In [86]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
from sklearn.metrics import r2_score
from datetime import datetime
import gc
import copy
import re
import os
from openpyxl import load_workbook

# Preprocess
- 内外利差类（1、2、3、5、7、10）
- 其他政府监管类
- 资金策略类指标

In [153]:
names_in = ['内外利差类指标.xlsx', '其他政府监管类指标.xlsx', '资金策略类指标.xlsx']
names_out = ['内外利差类指标_1.xlsx', '内外利差类指标_2.xlsx', '内外利差类指标_3.xlsx', 
             '内外利差类指标_5.xlsx', '内外利差类指标_7.xlsx', '内外利差类指标_10.xlsx', 
             '其他政府监管类指标.xlsx', '资金策略类指标.xlsx']
fix = pd.read_excel('国债收益率.xlsx', skipfooter = 1)
fix.set_index('描述', inplace = True, drop = True)
new = pd.read_excel(names_in[0], skipfooter = 1)
new.set_index('描述', inplace = True, drop = True)
new = new.join(fix['中债国债到期收益率:1年'], how = 'outer')
new['中国-美国:1年国债收益率'] = new['中债国债到期收益率:1年'] - new.pop('美国:国债收益率:1年')
new['中国-德国:1年国债收益率'] = new['中债国债到期收益率:1年'] - new.pop('德国:国债收益率:1年')
new['中国-法国:1年国债收益率'] = new.pop('中债国债到期收益率:1年') - new.pop('法国:国债收益率:1年')
new = new.join(fix['中债国债到期收益率:2年'], how = 'outer')
new['中国-美国:2年国债收益率'] = new['中债国债到期收益率:2年'] - new.pop('美国:国债收益率:2年')
new['中国-德国:2年国债收益率'] = new['中债国债到期收益率:2年'] - new.pop('德国:国债收益率:2年')
new['中国-法国:2年国债收益率'] = new.pop('中债国债到期收益率:2年') - new.pop('法国:国债收益率:2年')
new = new.join(fix['中债国债到期收益率:3年'], how = 'outer')
new['中国-美国:3年国债收益率'] = new['中债国债到期收益率:3年'] - new.pop('美国:国债收益率:3年')
new['中国-德国:3年国债收益率'] = new['中债国债到期收益率:3年'] - new.pop('德国:国债收益率:3年')
new = new.join(fix['中债国债到期收益率:5年'], how = 'outer')
new['中国-美国:5年国债收益率'] = new['中债国债到期收益率:5年'] - new.pop('美国:国债收益率:5年')
new['中国-德国:5年国债收益率'] = new['中债国债到期收益率:5年'] - new.pop('德国:国债收益率:5年')
new['中国-法国:5年国债收益率'] = new['中债国债到期收益率:5年'] - new.pop('法国:国债收益率:5年')
new['中国-英国:5年国债收益率'] = new.pop('中债国债到期收益率:5年') - new.pop('英国:国债收益率:5年')
new = new.join(fix['中债国债到期收益率:7年'], how = 'outer')
new['中国-美国:7年国债收益率'] = new['中债国债到期收益率:7年'] - new.pop('美国:国债收益率:7年')
new['中国-德国:7年国债收益率'] = new['中债国债到期收益率:7年'] - new.pop('德国:国债收益率:7年')
new = new.join(fix['中债国债到期收益率:10年'], how = 'outer')
new['中国-美国:10年国债收益率'] = new['中债国债到期收益率:10年'] - new.pop('美国:国债收益率:10年')
new['中国-德国:10年国债收益率'] = new['中债国债到期收益率:10年'] - new.pop('德国:国债收益率:10年')
new['中国-法国:10年国债收益率'] = new['中债国债到期收益率:10年'] - new.pop('法国:国债收益率:10年')
new['中国-英国:10年国债收益率'] = new.pop('中债国债到期收益率:10年') - new.pop('英国:国债收益率:10年')
new.sort_index(inplace = True)
s = pd.Series([np.nan] * new.shape[1])
s.name = '数据来源: 中债估值中心'
new = new.append(s)
new[['中国-美国:1年国债收益率', '中国-德国:1年国债收益率', '中国-法国:1年国债收益率']].to_excel(names_out[0])
new[['中国-美国:2年国债收益率', '中国-德国:2年国债收益率', '中国-法国:2年国债收益率']].to_excel(names_out[1])
new[['中国-美国:3年国债收益率', '中国-德国:3年国债收益率']].to_excel(names_out[2])
new[['中国-美国:5年国债收益率', '中国-德国:5年国债收益率', '中国-法国:5年国债收益率', '中国-英国:5年国债收益率']].to_excel(names_out[3])
new[['中国-美国:7年国债收益率', '中国-德国:7年国债收益率']].to_excel(names_out[4])
new[['中国-美国:10年国债收益率', '中国-德国:10年国债收益率', '中国-法国:10年国债收益率', '中国-英国:10年国债收益率']].to_excel(names_out[5])

In [56]:
files = ['经济前景类指标.xlsx', '投资类指标.xlsx', '工业类指标.xlsx', '进出口类指标.xlsx', '房地产类指标.xlsx', 
         '消费类指标.xlsx', '价格指数类指标.xlsx', '发改委价格类指标.xlsx', '统计局生产资料类指标.xlsx', '统计局食品类指标.xlsx', 
         '资金利率类指标.xlsx', '广义流动性类指标.xlsx', '内外利差类指标_1.xlsx', '内外利差类指标_2.xlsx', '内外利差类指标_3.xlsx', 
         '内外利差类指标_5.xlsx', '内外利差类指标_7.xlsx', '内外利差类指标_10.xlsx', '汇率类指标.xlsx', '跨境资本流动类指标.xlsx', 
         '财政政策类指标.xlsx', '货币政策类指标.xlsx', '其他政府监管1类指标.xlsx', '其他政府监管2类指标.xlsx', '资金策略1类指标.xlsx', 
         '资金策略2类指标.xlsx', '行业监管类指标.xlsx']
ins = ['中债国债到期收益率:1年', '中债国债到期收益率:2年', '中债国债到期收益率:3年', '中债国债到期收益率:5年', '中债国债到期收益率:7年', '中债国债到期收益率:10年']

In [55]:
treasury = pd.read_excel('国债收益率.xlsx', skipfooter = 1)
treasury.set_index('描述', inplace = True)
treasury.head()

,中债国债到期收益率:1年,中债国债到期收益率:2年,中债国债到期收益率:3年,中债国债到期收益率:5年,中债国债到期收益率:7年,中债国债到期收益率:10年
描述,,,,,,
2002-01-04,2.5850,2.6563,2.7271,2.8674,3.0057,3.2096
2002-01-07,2.6009,2.6697,2.7380,2.8728,3.0055,3.2003
2002-01-08,1.9156,2.1578,2.3842,2.7890,3.1302,3.5225
2002-01-09,1.9040,2.1627,2.4036,2.8317,3.1884,3.5896
2002-01-10,1.8987,2.1559,2.3954,2.8216,3.1772,3.5784


# Factor Selecting

In [116]:
def loading_file(i, sheet = 0):
    print('Loading file {} in 27 ...'.format(i+1))
    data = pd.read_excel(files[i], skipfooter = 1, sheet_name = sheet)
    dates = data['描述']
    data.set_index('描述', inplace = True)
    if i == 22:
        data = data.join(y[name], how = 'outer')
        data[name + '-3个月SHIBOR'] = data[name] - data.pop('SHIBOR:3个月')
        data[name + '-FR007'] = data[name] - data.pop('回购定盘利率:7天(FR007)')
        data[name + '-GC007'] = data.pop(name) - data.pop('GC007:收盘价')
    if i == 24:
        data = data.join(y[name], how = 'outer')
        data[name + '-同业存单到期收益率(AAA+):1年'] = data[name] - data.pop('同业存单到期收益率(AAA+):1年')
        data[name + '-短期融资券到期收益率(AAA+):1年'] = data[name] - data.pop('短期融资券到期收益率(AAA+):1年')
        data[name + '-贷款基础利率(LPR):1年'] = data[name] - data.pop('贷款基础利率(LPR):1年')
        if num == '1':
            data[name + '-中债国开债到期收益率:1年'] = data[name] - data.pop('中债国开债到期收益率:1年')
            data[name + '-中债商业银行普通债到期收益率(AAA):1年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):1年')
            data[name + '-中债企业债到期收益率(AAA):1年'] = data[name] - data.pop('中债企业债到期收益率(AAA):1年')
            data[name + '-中债铁道债到期收益率:1年'] = data[name] - data.pop('中债铁道债到期收益率:1年')
            data[name + '-中债地方政府债到期收益率(AAA):1年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):1年')
            data = data[[name + '-中债国开债到期收益率:1年', name + '-中债商业银行普通债到期收益率(AAA):1年', 
                         name + '-中债企业债到期收益率(AAA):1年', name + '-中债铁道债到期收益率:1年',
                         name + '-中债地方政府债到期收益率(AAA):1年', name + '-同业存单到期收益率(AAA+):1年', 
                         name + '-短期融资券到期收益率(AAA+):1年', name + '-贷款基础利率(LPR):1年']]
        if num == '2':
            data[name + '-中债国开债到期收益率:2年'] = data[name] - data.pop('中债国开债到期收益率:2年')
            data[name + '-中债商业银行普通债到期收益率(AAA):2年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):2年')
            data[name + '-中债企业债到期收益率(AAA):2年'] = data[name] - data.pop('中债企业债到期收益率(AAA):2年')
            data[name + '-中债铁道债到期收益率:2年'] = data[name] - data.pop('中债铁道债到期收益率:2年')
            data[name + '-中债地方政府债到期收益率(AAA):2年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):2年')
            data = data[[name + '-中债国开债到期收益率:2年', name + '-中债商业银行普通债到期收益率(AAA):2年', 
                         name + '-中债企业债到期收益率(AAA):2年', name + '-中债铁道债到期收益率:2年',
                         name + '-中债地方政府债到期收益率(AAA):2年', name + '-同业存单到期收益率(AAA+):1年', 
                         name + '-短期融资券到期收益率(AAA+):1年', name + '-贷款基础利率(LPR):1年']]
        if num == '3':
            data[name + '-中债国开债到期收益率:3年'] = data[name] - data.pop('中债国开债到期收益率:3年')
            data[name + '-中债商业银行普通债到期收益率(AAA):3年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):3年')
            data[name + '-中债企业债到期收益率(AAA):3年'] = data[name] - data.pop('中债企业债到期收益率(AAA):3年')
            data[name + '-中债铁道债到期收益率:3年'] = data[name] - data.pop('中债铁道债到期收益率:3年')
            data[name + '-中债地方政府债到期收益率(AAA):3年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):3年')
            data = data[[name + '-中债国开债到期收益率:3年', name + '-中债商业银行普通债到期收益率(AAA):3年', 
                         name + '-中债企业债到期收益率(AAA):3年', name + '-中债铁道债到期收益率:3年',
                         name + '-中债地方政府债到期收益率(AAA):3年', name + '-同业存单到期收益率(AAA+):1年', 
                         name + '-短期融资券到期收益率(AAA+):1年', name + '-贷款基础利率(LPR):1年']]
        if num == '5':
            data[name + '-中债国开债到期收益率:5年'] = data[name] - data.pop('中债国开债到期收益率:5年')
            data[name + '-中债商业银行普通债到期收益率(AAA):5年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):5年')
            data[name + '-中债企业债到期收益率(AAA):5年'] = data[name] - data.pop('中债企业债到期收益率(AAA):5年')
            data[name + '-中债铁道债到期收益率:5年'] = data[name] - data.pop('中债铁道债到期收益率:5年')
            data[name + '-中债地方政府债到期收益率(AAA):5年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):5年')
            data = data[[name + '-中债国开债到期收益率:5年', name + '-中债商业银行普通债到期收益率(AAA):5年', 
                         name + '-中债企业债到期收益率(AAA):5年', name + '-中债铁道债到期收益率:5年',
                         name + '-中债地方政府债到期收益率(AAA):5年', name + '-同业存单到期收益率(AAA+):1年', 
                         name + '-短期融资券到期收益率(AAA+):1年', name + '-贷款基础利率(LPR):1年']]
        if num == '7':
            data[name + '-中债国开债到期收益率:7年'] = data[name] - data.pop('中债国开债到期收益率:7年')
            data[name + '-中债商业银行普通债到期收益率(AAA):7年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):7年')
            data[name + '-中债企业债到期收益率(AAA):7年'] = data[name] - data.pop('中债企业债到期收益率(AAA):7年')
            data[name + '-中债铁道债到期收益率:7年'] = data[name] - data.pop('中债铁道债到期收益率:7年')
            data[name + '-中债地方政府债到期收益率(AAA):7年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):7年')
            data = data[[name + '-中债国开债到期收益率:7年', name + '-中债商业银行普通债到期收益率(AAA):7年', 
                         name + '-中债企业债到期收益率(AAA):7年', name + '-中债铁道债到期收益率:7年',
                         name + '-中债地方政府债到期收益率(AAA):7年', name + '-同业存单到期收益率(AAA+):1年', 
                         name + '-短期融资券到期收益率(AAA+):1年', name + '-贷款基础利率(LPR):1年']]
        if num == '10':
            data[name + '-中债国开债到期收益率:10年'] = data[name] - data.pop('中债国开债到期收益率:10年')
            data[name + '-中债商业银行普通债到期收益率(AAA):10年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):10年')
            data[name + '-中债企业债到期收益率(AAA):10年'] = data[name] - data.pop('中债企业债到期收益率(AAA):10年')
            data[name + '-中债铁道债到期收益率:10年'] = data[name] - data.pop('中债铁道债到期收益率:10年')
            data[name + '-中债地方政府债到期收益率(AAA):10年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):10年')
            data = data[[name + '-中债国开债到期收益率:10年', name + '-中债商业银行普通债到期收益率(AAA):10年', 
                         name + '-中债企业债到期收益率(AAA):10年', name + '-中债铁道债到期收益率:10年',
                         name + '-中债地方政府债到期收益率(AAA):10年', name + '-同业存单到期收益率(AAA+):1年', 
                         name + '-短期融资券到期收益率(AAA+):1年', name + '-贷款基础利率(LPR):1年']]
    return data

def ADFtest(data):
    alls = {'时间序列(差分处理)': [],
           'ADF值': [],
           '1%临界值': [],
           '5%临界值': [],
           '是否平稳': []}
    for col in data.columns:
        alls['时间序列(差分处理)'].append(col)
        data_rmna = data[col].dropna().diff(1).dropna()
        adftest = adfuller(data_rmna, autolag = 'AIC')
        alls['ADF值'].append(adftest[0])
        alls['1%临界值'].append(adftest[4]['1%'])
        alls['5%临界值'].append(adftest[4]['5%'])
        if adftest[0] < adftest[4]['1%']:
            alls['是否平稳'].append('平稳')
        else:
            alls['是否平稳'].append('非平稳')
    alls = pd.DataFrame.from_dict(alls)
    alls = alls[['时间序列(差分处理)', 'ADF值', '1%临界值', '5%临界值', '是否平稳']]      
    return alls

def quater(data):
    q = (data.month // 3) + 1
    return q

def cointest(data, period):
    alls = {'指标变化率的时间序列': [], 
           'P值': [],
           '是否协整': []}
    if period == 'special day':
        for col in data.columns:
            target = y.join(data[col], how = 'outer')
            target[name].fillna(inplace = True, method = 'ffill', limit = 31)
            target.dropna(inplace = True)
            cotest = coint(target[col], target[name])
            alls['指标变化率的时间序列'].append(col)
            alls['P值'].append(cotest[1])
            if cotest[1] > 0.05:
                alls['是否协整'].append('否')
            else:
                alls['是否协整'].append('是')
    else:
        if period == 'day':
            ys = y
            datanew = data 
        if period == 'month':
            datanew = copy.deepcopy(data)
            datanew['year'] = datanew.index.year
            datanew['month'] = datanew.index.month
            datanew.set_index(['year', 'month'], drop = True, inplace = True)
            ys = copy.deepcopy(y)
            ys['year'] = ys.index.year
            ys['month'] = ys.index.month
            ys.drop_duplicates(['year', 'month'], keep = 'last', inplace = True)
            ys.set_index(['year', 'month'], drop = True, inplace = True) 
        if period == 'quater':
            datanew = copy.deepcopy(data)
            datanew['year'] = datanew.index.year
            datanew['quater'] = quater(datanew.index)
            datanew.set_index(['year', 'quater'], drop = True, inplace = True)
            ys = copy.deepcopy(y)
            ys['year'] = ys.index.year
            ys['quater'] = quater(ys.index)
            ys.drop_duplicates(['year', 'quater'], keep = 'last', inplace = True)
            ys.set_index(['year', 'quater'], drop = True, inplace = True)
        for col in data.columns:
            target = ys.join(datanew[col], how = 'inner')
            target.dropna(inplace = True)
            cotest = coint(target[col], target[name])
            alls['指标变化率的时间序列'].append(col)
            alls['P值'].append(cotest[1])
            if cotest[1] > 0.05:
                alls['是否协整'].append('否')
            else:
                alls['是否协整'].append('是')
    alls = pd.DataFrame.from_dict(alls)
    alls = alls[['指标变化率的时间序列', 'P值', '是否协整']]
    return alls

def reg(data, period):
    alls = {'指标变化率的时间序列': [], 
           'coef': [],
           'Prob(F-statistic)': [], 
           'R-squared': [],
           'No.Observations': []}
    if period == 'special day':
        for col in data.columns:
            target = y.join(data[col], how = 'outer')
            target[name].fillna(inplace = True, method = 'ffill', limit = 31)
            target.dropna(inplace = True)
            model = sm.OLS(endog = target[name], exog = sm.add_constant(target[col]))
            res = model.fit()
            r2 = r2_score(target[name], res.fittedvalues)
            alls['指标变化率的时间序列'].append(col)
            alls['coef'].append(res.params[1])
            alls['Prob(F-statistic)'].append(res.pvalues[1])
            alls['R-squared'].append(r2)
            alls['No.Observations'].append(target.shape[0])
    else:
        if period == 'day':
            ys = y
            datanew = data 
        if period == 'month':
            datanew = copy.deepcopy(data)
            datanew['year'] = datanew.index.year
            datanew['month'] = datanew.index.month
            datanew.set_index(['year', 'month'], drop = True, inplace = True)
            ys = copy.deepcopy(y)
            ys['year'] = ys.index.year
            ys['month'] = ys.index.month
            ys.drop_duplicates(['year', 'month'], keep = 'last', inplace = True)
            ys.set_index(['year', 'month'], drop = True, inplace = True) 
        if period == 'quater':
            datanew = copy.deepcopy(data)
            datanew['year'] = datanew.index.year
            datanew['quater'] = quater(datanew.index)
            datanew.set_index(['year', 'quater'], drop = True, inplace = True)
            ys = copy.deepcopy(y)
            ys['year'] = ys.index.year
            ys['quater'] = quater(ys.index)
            ys.drop_duplicates(['year', 'quater'], keep = 'last', inplace = True)
            ys.set_index(['year', 'quater'], drop = True, inplace = True)
        for col in data.columns:
            target = ys.join(datanew[col], how = 'inner')
            target.dropna(inplace = True)
            model = sm.OLS(endog = target[name], exog = sm.add_constant(target[col]))
            res = model.fit()
            r2 = r2_score(target[name], res.fittedvalues)
            alls['指标变化率的时间序列'].append(col)
            alls['coef'].append(res.params[1])
            alls['Prob(F-statistic)'].append(res.pvalues[1])
            alls['R-squared'].append(r2)
            alls['No.Observations'].append(target.shape[0])
    alls = pd.DataFrame.from_dict(alls)
    alls = alls[['指标变化率的时间序列', 'coef', 'Prob(F-statistic)', 'R-squared', 'No.Observations']]
    return alls

def csv_for_en(data, cols, filename, period):
    name = list(y.columns)[0]
    if period == 'special day':
        datanew = data[cols].join(y[name], how = 'outer')
        datanew[name].fillna(inplace = True, method = 'ffill', limit = 31)
        datanew.dropna(inplace = True)
    else:
        if period == 'day':
            ys = y
            datanew = data 
        if period == 'month':
            datanew = copy.deepcopy(data)
            datanew['year'] = datanew.index.year
            datanew['month'] = datanew.index.month
            datanew.set_index(['year', 'month'], drop = True, inplace = True)
            ys = copy.deepcopy(y)
            ys['year'] = ys.index.year
            ys['month'] = ys.index.month
            ys.drop_duplicates(['year', 'month'], keep = 'last', inplace = True)
            ys.set_index(['year', 'month'], drop = True, inplace = True) 
        if period == 'quater':
            datanew = copy.deepcopy(data)
            datanew['year'] = datanew.index.year
            datanew['quater'] = quater(datanew.index)
            datanew.set_index(['year', 'quater'], drop = True, inplace = True)
            ys = copy.deepcopy(y)
            ys['year'] = ys.index.year
            ys['quater'] = quater(ys.index)
            ys.drop_duplicates(['year', 'quater'], keep = 'last', inplace = True)
            ys.set_index(['year', 'quater'], drop = True, inplace = True)
        datanew = datanew[cols].join(ys[name], how = 'inner')
        datanew.dropna(inplace = True)
#     datanew.to_csv(filename, encoding = 'utf_8_sig', index = False, header = None)
    return datanew
    
def gen_results(adf, coin, results):
    adf_ = adf.set_index('时间序列(差分处理)', drop = True)
    coin_ = coin.set_index('指标变化率的时间序列', drop = True)
    results_ = results.set_index('指标变化率的时间序列', drop = True)
    alls = results_[['coef', 'Prob(F-statistic)', 'R-squared']].join(adf_['是否平稳'], how = 'outer')
    alls = alls.join(coin_['是否协整'])
    return alls
def auto(i, period1):
    data = loading_file(i)
    adf = ADFtest(data)
    coin = cointest(data, period = period1)
    results = reg(data, period = period1)
    fin = gen_results(adf, coin, results)
    return fin

In [106]:
def excelAddSheet(dataframe, outfile, name, ind = True):
    writer = pd.ExcelWriter(outfile, enginge='openpyxl')
    if os.path.exists(outfile) != True:
        dataframe.to_excel(writer, name, index = ind)
    else:
        book = load_workbook(writer.path)
        writer.book = book
        dataframe.to_excel(excel_writer = writer, sheet_name = name, index = ind)
    writer.save()
    writer.close()

In [95]:
out_names = []
for file in files:
    out = file.split('.')[0]
    out_names.append(out)
for j in range(6):
    print('treasury', j)
    name = ins[j]
    num = re.findall('\d+', name)[0]
    y = pd.DataFrame(treasury[name])
    for i in range(27):
        if (i == 7 or i == 10 or i == 12 or i == 13 or i == 14 or i == 15 or i == 16 or i == 17 or i == 18 or i == 21 or i == 22 or i == 24):
            final = auto(i, 'day')
        if (i == 8 or i == 9):
            final = auto(i, 'special day')
        if i == 26:
            final = auto(i, 'quater')
        if (i == 0 or i == 1 or i == 2 or i == 3 or i == 4 or i == 5 or i == 6 or i == 11 or i == 19 or i == 20 or i == 23 or i == 25):
            final = auto(i, 'month')
        excelAddSheet(final, num + '_result.xlsx', out_names[i])

treasury 0
Loading file 1 in 27 ...
Loading file 2 in 27 ...
Loading file 3 in 27 ...
Loading file 4 in 27 ...
Loading file 5 in 27 ...
Loading file 6 in 27 ...
Loading file 7 in 27 ...
Loading file 8 in 27 ...
Loading file 9 in 27 ...
Loading file 10 in 27 ...
Loading file 11 in 27 ...
Loading file 12 in 27 ...
Loading file 13 in 27 ...
Loading file 14 in 27 ...
Loading file 15 in 27 ...
Loading file 16 in 27 ...
Loading file 17 in 27 ...
Loading file 18 in 27 ...
Loading file 19 in 27 ...
Loading file 20 in 27 ...
Loading file 21 in 27 ...
Loading file 22 in 27 ...
Loading file 23 in 27 ...
Loading file 24 in 27 ...
Loading file 25 in 27 ...
Loading file 26 in 27 ...
Loading file 27 in 27 ...
treasury 1
Loading file 1 in 27 ...
Loading file 2 in 27 ...
Loading file 3 in 27 ...
Loading file 4 in 27 ...
Loading file 5 in 27 ...
Loading file 6 in 27 ...
Loading file 7 in 27 ...
Loading file 8 in 27 ...
Loading file 9 in 27 ...
Loading file 10 in 27 ...
Loading file 11 in 27 ...
Loading 

# 输出给R

In [125]:
col = [['PMI', 'PMI:生产', 'PMI:新订单', 'PMI:新出口订单', 'PMI:在手订单', 'PMI:产成品库存',
       'PMI:采购量', 'PMI:进口', 'PMI:主要原材料购进价格', 'PMI:原材料库存',
       'PMI:从业人员', 'PMI:供货商配送时间', '财新中国PMI', '财新中国服务业PMI:经营活动指数'], 
      ['固定资产投资资金来源:国内贷款:累计同比',  '固定资产投资资金来源:国家预算内资金:累计同比',
       '固定资产投资资金来源:自筹资金:累计同比', '固定资产投资完成额:累计同比', '房地产开发投资完成额:累计同比',
       '新增固定资产投资完成额:累计同比', '固定资产投资完成额:制造业:累计同比', '固定资产投资完成额:基础设施建设投资:累计同比'],
      ['工业增加值:累计同比', '工业增加值:汽车制造业:累计同比',
       '工业企业:利润总额:累计同比', '工业企业:利润总额:当月同比',
       '工业企业:主营业务收入:累计同比', '工业企业:产成品存货:累计同比', '工业企业:资产负债率'],
       ['出口金额:当月同比', '进口金额:当月同比', '贸易差额:当月同比', '贸易差额:当月值', '美国:进出口金额:累计同比',
       '日本:进出口金额:累计同比', '欧盟:进出口金额:累计同比', '美国:贸易差额:当月值', '欧盟:贸易差额:当月值',
       '日本:贸易差额:当月值'],
       ['房地产开发投资完成额:累计同比', '待开发土地面积:累计同比', '本年购置土地面积:累计同比', '本年土地成交价款:累计同比',
       '土地购置费:累计同比', '房屋施工面积:累计同比', '房屋新开工面积:累计同比', '房屋竣工面积:累计同比',
       '商品房销售面积:累计同比', '商品房销售额:累计同比', '商品房待售面积:累计同比', '商品房待售面积:累计值'],
       ['社会消费品零售总额:累计同比', '社会消费品零售总额:实际累计同比', '社会消费品零售总额:城镇:累计同比',
       '社会消费品零售总额:乡村:累计同比', '消费者信心指数(月)'],
       ['CPI:当月同比', 'CPI:食品:当月同比', 'CPI:非食品:当月同比', 'CPI:不包括食品和能源(核心CPI):当月同比',
       'CPI:不包括鲜菜和鲜果:当月同比', 'CPI:消费品:当月同比', 'CPI:服务:当月同比',
       'CPI:食品烟酒:畜肉类:猪肉:当月同比', 'PPI:全部工业品:当月同比', 'PPI:生产资料:当月同比',
       'PPI:生产资料:采掘工业:当月同比', 'PPI:生产资料:原材料工业:当月同比', 'PPI:生产资料:加工工业:当月同比',
       'PPI:生活资料:当月同比', 'PPI:生活资料:食品类:当月同比', 'PPI:生活资料:衣着类:当月同比',
       'PPI:生活资料:一般日用品类:当月同比', 'PPI:生活资料:耐用消费品类:当月同比'],
       ['36个城市平均零售价:晚籼米', '36个城市平均零售价:粳米', '36个城市平均零售价:面粉:富强粉',
       '36个城市平均零售价:面粉:标准粉', '36个城市平均零售价:花生油:桶装', '36个城市平均零售价:菜籽油:桶装',
       '36个城市平均零售价:豆油:桶装', '36个城市平均零售价:猪肉', '36个城市平均零售价:牛肉', '36个城市平均零售价:羊肉',
       '36个城市平均零售价:鸡肉', '36个城市平均零售价:鸡蛋', '36个城市平均零售价:草鱼', '36个城市平均零售价:鲢鱼',
       '36个大中城市平均零售价:带鱼', '36个大中城市平均零售价:芹菜', '36个大中城市平均零售价:大白菜',
       '36个大中城市平均零售价:萝卜', '36个大中城市平均零售价:土豆', '36个大中城市平均零售价:胡萝卜',
       '36个大中城市平均零售价:尖椒', '36个大中城市平均零售价:圆白菜', '36个大中城市平均零售价:豆角',
       '36个大中城市平均零售价:蒜苔', '36个大中城市平均零售价:韭菜', '36个大中城市平均零售价:青椒',
       '36个大中城市平均零售价:黄瓜', '36个大中城市平均零售价:西红柿', '36个大中城市平均零售价:油菜',
       '36个大中城市平均零售价:茄子'],
       ['市场价:螺纹钢:HRB400 Φ16-25mm:全国', '市场价:线材:HPB300 Φ6.5mm:全国',
       '市场价:普通中板:Q235 20mm:全国', '市场价:热轧普通薄板:Q235 3mm:全国',
       '市场价:无缝钢管:20# 219*6:全国', '市场价:角钢:5#:全国', '市场价:电解铜:1#:全国',
       '市场价:铝锭:A00:全国', '市场价:铅锭:1#:全国', '市场价:锌锭:0#:全国', '市场价:硫酸(98%):全国',
       '市场价:液碱(32%):全国', '市场价:甲醇(优等品):全国', '市场价:石油苯(工业级):全国',
       '市场价:苯乙烯(一级品):全国', '市场价:LLDPE(7042):全国', '市场价:聚丙烯(T30S):全国',
       '市场价:聚氯乙烯(SG5):全国', '市场价:顺丁橡胶(BR9000):全国', '市场价:涤纶长丝(FDY150D/96F):全国',
       '市场价:液化天然气LNG:全国', '市场价:液化气LPG:全国', '市场价:汽油(97#):全国', '市场价:汽油(93#):全国',
       '市场价:柴油(0#):全国', '市场价:石蜡(58#半):全国', '市场价:无烟煤(2号洗中块):全国',
       '市场价:普通混煤(Q4500):全国', '市场价:山西大混(Q5000):全国', '市场价:山西优混(Q5500):全国',
       '市场价:大同混煤(Q5800):全国', '市场价:1/3焦煤:全国', '市场价:二级冶金焦:全国',
       '市场价:复合硅酸盐水泥:P.C32.5 袋装:全国', '市场价:普通硅酸盐水泥:P.O42.5 散装:全国',
       '市场价:浮法平板玻璃:4.8/5mm:全国', '市场价:稻米:粳稻米:全国', '市场价:小麦:国标三等:全国',
       '市场价:玉米:黄玉米二等:全国', '市场价:棉花:皮棉 白棉三级:全国', '市场价:生猪:外三元:全国', '市场价:大豆:黄豆:全国',
       '市场价:豆粕:粗蛋白含量≥43%:全国', '市场价:花生:油料花生米:全国', '市场价:尿素(小颗料):全国',
       '市场价:硫酸钾复合肥:全国', '市场价:草甘膦(95%原药):全国', '市场价:人造板:1220*2440*15mm:全国',
       '市场价:纸浆:漂白化学浆:全国', '市场价:瓦楞纸:高强:全国'],
       ['50个城市平均价:面粉:标准粉', '50个城市平均价:面粉:富强粉', '50个城市平均价:大米', '50个城市平均价:豆制品:豆腐',
       '50个城市平均价:花生油:压榨一级', '50个城市平均价:大豆油:5L桶装', '50个城市平均价:菜籽油:一级散装',
       '50个城市平均价:猪肉:猪肉后臀尖(后腿肉)', '50个城市平均价:猪肉:五花肉', '50个城市平均价:牛肉:腿肉',
       '50个城市平均价:羊肉:腿肉', '50个城市平均价:鸡:白条鸡', '50个城市平均价:鸡:鸡胸肉', '50个城市平均价:鸭',
       '50个城市平均价:鸡蛋', '50个城市平均价:活鲤鱼', '50个城市平均价:活草鱼', '50个城市平均价:带鱼',
       '50个城市平均价:大白菜', '50个城市平均价:油菜', '50个城市平均价:芹菜', '50个城市平均价:黄瓜',
       '50个城市平均价:西红柿', '50个城市平均价:豆角', '50个城市平均价:土豆', '50个城市平均价:苹果:富士',
       '50个城市平均价:香蕉:国产'],
       ['银行间质押式回购加权利率:1天', '银行间质押式回购加权利率:7天', '银行间质押式回购加权利率:14天',
       '银行间质押式回购加权利率:21天', '银行间质押式回购加权利率:1个月', '银行间质押式回购加权利率:2个月',
       '银行间质押式回购加权利率:3个月', '银行间质押式回购加权利率:4个月', '银行间质押式回购加权利率:6个月',
       '存款类机构质押式回购加权利率:1天',
       '存款类机构质押式回购加权利率:7天', '存款类机构质押式回购加权利率:14天', '存款类机构质押式回购加权利率:21天',
       '存款类机构质押式回购加权利率:1个月', '存款类机构质押式回购加权利率:2个月', '存款类机构质押式回购加权利率:3个月',
       'SHIBOR:隔夜', 'SHIBOR:1周', 'SHIBOR:2周',
       'SHIBOR:1个月', 'SHIBOR:3个月', 'SHIBOR:6个月', 'SHIBOR:9个月', 'SHIBOR:1年',
       'GC001:加权平均', 'GC002:加权平均', 'GC003:加权平均', 'GC004:加权平均', 'GC007:加权平均',
       'GC014:加权平均', 'GC028:加权平均', 'GC091:加权平均', 'GC182:加权平均'], 
       ['M1:同比', 'M2:同比', '社会融资规模:当月值', '社会融资规模:累计值', '社会融资规模:新增人民币贷款:当月值',
       '社会融资规模:新增人民币贷款:累计值', '金融机构:短期贷款余额', '金融机构:中长期贷款余额', '金融机构:各项贷款余额',
       '金融机构:各项贷款余额:同比', '金融机构:各项存款余额', '金融机构:各项存款余额:同比',
       '金融机构:新增人民币贷款:短期贷款及票据融资:当月值', '金融机构:新增人民币贷款:票据融资:当月值',
       '金融机构:新增人民币贷款:中长期:当月值', '金融机构:新增人民币贷款:居民户:当月值',
       '金融机构:新增人民币贷款:居民户:短期:当月值', '金融机构:新增人民币贷款:居民户:中长期:当月值',
       '金融机构:新增人民币贷款:非金融性公司及其他部门:当月值', '金融机构:新增人民币贷款:非金融性公司:短期:当月值',
       '金融机构:新增人民币贷款:非金融性公司:中长期:当月值'], 
       ['中国-美国:1年国债收益率', '中国-德国:1年国债收益率', '中国-法国:1年国债收益率'], 
       ['中国-美国:2年国债收益率', '中国-德国:2年国债收益率', '中国-法国:2年国债收益率'],
       ['中国-美国:3年国债收益率', '中国-德国:3年国债收益率'],
       ['中国-美国:5年国债收益率', '中国-德国:5年国债收益率', '中国-法国:5年国债收益率', '中国-英国:5年国债收益率'],
       ['中国-美国:7年国债收益率', '中国-德国:7年国债收益率'],
       ['中国-美国:10年国债收益率', '中国-德国:10年国债收益率', '中国-法国:10年国债收益率', '中国-英国:10年国债收益率'],
       ['美元指数', '欧元兑美元', '美元兑日元', '英镑兑美元', '美元兑人民币元'], 
       ['官方储备资产:外汇储备', '货币当局:国外资产:外汇(中央银行外汇占款)'], 
       ['公共财政收入:当月同比', '公共财政支出:当月同比', '公共财政支出:累计同比', '公共财政收入:累计同比', 
       '债券发行量:政府债券:累计值', '债券发行量:地方政府债:累计值', '债券发行量:政策性银行债:累计值'],
       ['公开市场操作:货币净投放', '公开市场操作:货币投放', '公开市场操作:货币回笼',
       '逆回购:到期量', '人民币存款准备金率:中小型存款类金融机构(月)', '人民币存款准备金率:大型存款类金融机构(月)',
       '超额存款准备金率(超储率):金融机构', '超额存款准备金率(超储率):农村信用社', '货币当局:国外资产:外汇(中央银行外汇占款)',
       '货币当局:对其他存款性公司债权', '货币当局:政府存款'],
       [],
       ['信托业:未来一年到期规模:合计', '信托业:未来一年到期规模:集合信托', '信托业:未来一年到期规模:单一信托',
       '信托业:未来一年到期规模:财产权信托', '保险公司:银行存款和债券:占资金运用余额比例', '保险公司:保险资金运用余额:投资:债券投资',
       '基金管理公司管理资产规模:公募基金', '公募基金份额:开放式基金:债券型', '公募基金净值:开放式基金:债券型',
       '证券投资基金成交金额:当月值', '银行理财产品资金余额', '中债:债券托管量:国债:商业银行', '中债:债券托管量:国债:保险机构',
       '待购回债券余额', '中债:债券托管量:国债:合计', '债券市场发行债券:同业存单', '债券市场托管余额:同业存单',
       '其他存款性公司:对其他金融机构债权'],
       [],
      ['居民活期存款/各项贷款', '居民活期存款/居住中长期贷款', '居民活期存款/非金融企业及机关团体各项贷款',
        '非金融企业活期存款/各项贷款', '非金融企业活期存款/居住中长期贷款', '非金融企业活期存款/非金融企业及机关团体各项贷款',
        '债券投资/资金运用总计', '各项贷款/债券投资'], 
      ['商业银行:流动性比例', '商业银行:存贷比', '商业银行:人民币超额备付金率', '商业银行:资本充足率', '商业银行:净息差',
       '不良贷款余额:商业银行', '商业银行:拨备覆盖率']]
for j in range(6):
    print('treasury', j)
    name = ins[j]
    num = re.findall('\d+', name)[0]
    y = pd.DataFrame(treasury[name])
    for i in range(27):
        data = loading_file(i)
        if (i == 0 or i == 1 or i == 2 or i == 3 or i == 4 or i == 5 or i == 6 or i == 11 or i == 19 or i == 20 or i == 23 or i == 25):  
            data_r = csv_for_en(data, col[i], str(i) + '.csv', 'month')
        if (i == 8 or i == 9):
            data_r = csv_for_en(data, col[i], str(i) + '.csv', 'special day')
        if (i == 7 or i == 10 or i == 12 or i == 13 or i == 14 or i == 15 or i == 16 or i == 17 or i == 18 or i == 21):
            data_r = csv_for_en(data, col[i], str(i) + '.csv', 'day')
        if (i == 22 or i == 24):
            data_r = csv_for_en(data, list(data.columns), str(i) + '.csv', 'day')
        if i == 26:
            data_r = csv_for_en(data, col[i], str(i) + '.csv', 'quater')
        excelAddSheet(data_r, num + '_r.xlsx', out_names[i], ind = False)

treasury 0
Loading file 1 in 27 ...
Loading file 2 in 27 ...
Loading file 3 in 27 ...
Loading file 4 in 27 ...
Loading file 5 in 27 ...
Loading file 6 in 27 ...
Loading file 7 in 27 ...
Loading file 8 in 27 ...
Loading file 9 in 27 ...
Loading file 10 in 27 ...
Loading file 11 in 27 ...
Loading file 12 in 27 ...
Loading file 13 in 27 ...
Loading file 14 in 27 ...
Loading file 15 in 27 ...
Loading file 16 in 27 ...
Loading file 17 in 27 ...
Loading file 18 in 27 ...
Loading file 19 in 27 ...
Loading file 20 in 27 ...
Loading file 21 in 27 ...
Loading file 22 in 27 ...
Loading file 23 in 27 ...
Loading file 24 in 27 ...
Loading file 25 in 27 ...
Loading file 26 in 27 ...
Loading file 27 in 27 ...
treasury 1
Loading file 1 in 27 ...
Loading file 2 in 27 ...
Loading file 3 in 27 ...
Loading file 4 in 27 ...
Loading file 5 in 27 ...
Loading file 6 in 27 ...
Loading file 7 in 27 ...
Loading file 8 in 27 ...
Loading file 9 in 27 ...
Loading file 10 in 27 ...
Loading file 11 in 27 ...
Loading 

# 经济前景类指标 1

In [62]:
fin1.to_csv('fin.csv', encoding = 'utf_8_sig')

In [6]:
factor1 = loading_file(0)
factor1

Loading file 1 in 20 ...


,PMI,PMI:生产,PMI:新订单,PMI:新出口订单,PMI:在手订单,PMI:产成品库存,PMI:采购量,PMI:进口,PMI:出厂价格,PMI:主要原材料购进价格,PMI:原材料库存,PMI:从业人员,PMI:供货商配送时间,PMI:生产经营活动预期,财新中国PMI,财新中国服务业PMI:经营活动指数,财新中国综合PMI:产出指数
描述,,,,,,,,,,,,,,,,,
2005-01-01,54.7,57.6,60.7,58.3,46.6,46.6,59.0,54.3,NaN,63.7,50.1,48.5,50.7,NaN,NaN,NaN,NaN
2005-02-01,54.5,55.6,61.5,59.9,49.8,47.5,60.0,55.2,NaN,64.8,47.8,49.7,50.6,NaN,NaN,NaN,NaN
2005-03-01,57.9,65.2,63.5,63.9,50.6,46.9,62.8,56.9,NaN,71.5,47.7,51.7,50.2,NaN,NaN,NaN,NaN
2005-04-01,56.7,62.7,61.3,60.3,50.9,46.5,61.0,54.2,NaN,65.2,48.4,51.8,50.4,NaN,NaN,NaN,NaN
2005-05-01,52.9,57.0,55.6,58.2,47.9,45.8,54.1,49.4,NaN,53.5,45.2,50.0,50.5,NaN,NaN,NaN,NaN
2005-06-01,51.7,55.7,53.4,56.1,48.4,48.1,53.2,49.8,NaN,49.8,44.2,49.9,50.7,NaN,NaN,NaN,NaN
2005-07-01,51.1,53.3,53.2,52.8,46.4,46.6,51.3,50.9,NaN,50.6,43.7,50.8,51.7,NaN,NaN,NaN,NaN
2005-08-01,52.6,55.8,55.0,52.9,47.7,44.2,53.3,49.1,NaN,53.7,44.3,51.8,51.1,NaN,NaN,NaN,NaN
2005-09-01,55.1,61.0,58.4,57.2,49.4,45.8,58.1,54.3,NaN,52.8,48.0,51.3,51.5,NaN,50.9,NaN,NaN


In [10]:
fin1, results1 = auto(0, 'month')
results1

Loading file 1 in 20 ...


,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,PMI,-0.032908,1.375653e-01,0.013729,162
1,PMI:生产,-0.030348,3.857900e-02,0.026473,162
2,PMI:新订单,-0.026388,4.218991e-02,0.025546,162
3,PMI:新出口订单,-0.007513,5.641890e-01,0.002082,162
4,PMI:在手订单,-0.032318,9.361757e-02,0.017470,162
5,PMI:产成品库存,0.137959,1.299302e-05,0.112361,162
6,PMI:采购量,-0.023289,8.702388e-02,0.018193,162
7,PMI:进口,0.007982,6.511270e-01,0.001281,162
8,PMI:出厂价格,0.026467,3.439457e-01,0.032038,30
9,PMI:主要原材料购进价格,0.008255,2.063290e-01,0.009962,162


In [12]:
fin1

,coef,Prob(F-statistic),R-squared,是否平稳,是否协整
PMI,-0.032908,1.375653e-01,0.013729,平稳,否
PMI:生产,-0.030348,3.857900e-02,0.026473,平稳,否
PMI:新订单,-0.026388,4.218991e-02,0.025546,平稳,否
PMI:新出口订单,-0.007513,5.641890e-01,0.002082,平稳,否
PMI:在手订单,-0.032318,9.361757e-02,0.017470,平稳,否
PMI:产成品库存,0.137959,1.299302e-05,0.112361,平稳,否
PMI:采购量,-0.023289,8.702388e-02,0.018193,平稳,否
PMI:进口,0.007982,6.511270e-01,0.001281,平稳,否
PMI:出厂价格,0.026467,3.439457e-01,0.032038,非平稳,否
PMI:主要原材料购进价格,0.008255,2.063290e-01,0.009962,平稳,否


In [125]:
col = ['PMI', 'PMI:生产', 'PMI:新订单', 'PMI:新出口订单', 'PMI:在手订单', 'PMI:产成品库存',
       'PMI:采购量', 'PMI:进口', 'PMI:主要原材料购进价格', 'PMI:原材料库存',
       'PMI:从业人员', 'PMI:供货商配送时间', '财新中国PMI', '财新中国服务业PMI:经营活动指数']
csv_for_en(factor1, col, '1.csv', 'month')

# 投资类指标 2

In [13]:
factor2 = loading_file(1)
factor2

Loading file 2 in 20 ...


,固定资产投资资金来源:国内贷款:累计同比,固定资产投资资金来源:国家预算内资金:累计同比,固定资产投资资金来源:自筹资金:累计同比,固定资产投资完成额:累计同比,房地产开发投资完成额:累计同比,新增固定资产投资完成额:累计同比,固定资产投资完成额:制造业:累计同比,固定资产投资完成额:基础设施建设投资:累计同比,固定资产投资完成额:基础设施建设投资(不含电力):累计同比
描述,,,,,,,,,
1992-02-01,NaN,NaN,NaN,56.7,NaN,NaN,NaN,NaN,NaN
1992-06-01,NaN,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN
1992-07-01,NaN,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN
1992-08-01,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN
1992-09-01,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN
1992-10-01,NaN,NaN,NaN,33.6,NaN,NaN,NaN,NaN,NaN
1992-11-01,NaN,NaN,NaN,33.9,NaN,NaN,NaN,NaN,NaN
1992-12-01,NaN,NaN,NaN,36.8,NaN,NaN,NaN,NaN,NaN
1993-02-01,NaN,NaN,NaN,64.1,NaN,NaN,NaN,NaN,NaN


In [19]:
fin2, results2 = auto(1, 'month')
results2

Loading file 2 in 20 ...


,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,固定资产投资资金来源:国内贷款:累计同比,-0.020127,5.394947e-09,0.177223,177
1,固定资产投资资金来源:国家预算内资金:累计同比,-0.018775,8.707202e-14,0.272996,177
2,固定资产投资资金来源:自筹资金:累计同比,-0.015172,2.095802e-05,0.098525,177
3,固定资产投资完成额:累计同比,-0.029869,3.115203e-06,0.115293,180
4,房地产开发投资完成额:累计同比,-0.000408,9.357463e-01,0.000037,180
5,新增固定资产投资完成额:累计同比,-0.016760,2.737477e-07,0.158148,156
6,固定资产投资完成额:制造业:累计同比,-0.009554,2.279880e-02,0.032786,158
7,固定资产投资完成额:基础设施建设投资:累计同比,-0.038095,8.138857e-11,0.243221,154
8,固定资产投资完成额:基础设施建设投资(不含电力):累计同比,0.053201,4.397426e-02,0.089046,46


In [20]:
fin2

,coef,Prob(F-statistic),R-squared,是否平稳,是否协整
固定资产投资资金来源:国内贷款:累计同比,-0.020127,5.394947e-09,0.177223,平稳,否
固定资产投资资金来源:国家预算内资金:累计同比,-0.018775,8.707202e-14,0.272996,平稳,否
固定资产投资资金来源:自筹资金:累计同比,-0.015172,2.095802e-05,0.098525,平稳,否
固定资产投资完成额:累计同比,-0.029869,3.115203e-06,0.115293,平稳,否
房地产开发投资完成额:累计同比,-0.000408,9.357463e-01,0.000037,平稳,否
新增固定资产投资完成额:累计同比,-0.016760,2.737477e-07,0.158148,平稳,否
固定资产投资完成额:制造业:累计同比,-0.009554,2.279880e-02,0.032786,平稳,否
固定资产投资完成额:基础设施建设投资:累计同比,-0.038095,8.138857e-11,0.243221,平稳,否
固定资产投资完成额:基础设施建设投资(不含电力):累计同比,0.053201,4.397426e-02,0.089046,平稳,否


In [21]:
col = ['固定资产投资资金来源:国内贷款:累计同比',
 '固定资产投资资金来源:国家预算内资金:累计同比',
 '固定资产投资资金来源:自筹资金:累计同比',
 '固定资产投资完成额:累计同比',
 '房地产开发投资完成额:累计同比',
 '新增固定资产投资完成额:累计同比',
 '固定资产投资完成额:制造业:累计同比',
 '固定资产投资完成额:基础设施建设投资:累计同比']
csv_for_en(factor2, col, '2.csv', 'month')

# 工业类指标 3

In [22]:
factor3 = loading_file(2)
factor3

Loading file 3 in 20 ...


,工业增加值:累计同比,工业增加值:采矿业:累计同比,工业增加值:制造业:累计同比,工业增加值:汽车制造业:累计同比,工业增加值:电力、燃气及水的生产和供应业:累计同比,工业企业:利润总额:累计同比,工业企业:利润总额:当月同比,工业企业:主营业务收入:累计同比,工业企业:产成品存货:累计同比,工业企业:资产负债率
描述,,,,,,,,,,
1996-02-01,NaN,NaN,NaN,NaN,NaN,-50.30,NaN,14.60,23.60,NaN
1996-03-01,NaN,NaN,NaN,NaN,NaN,-54.70,NaN,12.60,24.60,NaN
1996-04-01,NaN,NaN,NaN,NaN,NaN,-49.60,NaN,12.20,24.50,NaN
1996-05-01,NaN,NaN,NaN,NaN,NaN,-42.90,NaN,12.10,23.40,NaN
1996-06-01,NaN,NaN,NaN,NaN,NaN,-36.90,NaN,12.20,22.60,NaN
1996-07-01,NaN,NaN,NaN,NaN,NaN,-36.10,NaN,12.20,20.90,NaN
1996-08-01,NaN,NaN,NaN,NaN,NaN,-34.90,NaN,12.00,19.70,NaN
1996-09-01,NaN,NaN,NaN,NaN,NaN,-32.60,NaN,11.90,18.30,NaN
1996-10-01,NaN,NaN,NaN,NaN,NaN,-26.50,NaN,12.30,16.70,NaN


In [23]:
fin3, results3 = auto(2, 'month')
results3

Loading file 3 in 20 ...


,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,工业增加值:累计同比,-0.012617,2.328634e-01,0.007254,198
1,工业增加值:采矿业:累计同比,0.065697,2.433711e-02,0.090412,56
2,工业增加值:制造业:累计同比,0.303605,4.493329e-09,0.474073,56
3,工业增加值:汽车制造业:累计同比,0.077705,3.103308e-04,0.172922,71
4,工业增加值:电力、燃气及水的生产和供应业:累计同比,0.104738,7.150363e-05,0.255243,56
5,工业企业:利润总额:累计同比,-0.001867,4.717645e-01,0.003458,152
6,工业企业:利润总额:当月同比,0.023665,1.986440e-03,0.146174,63
7,工业企业:主营业务收入:累计同比,-0.004001,4.450666e-01,0.003893,152
8,工业企业:产成品存货:累计同比,0.020467,7.931679e-03,0.046054,152
9,工业企业:资产负债率,-0.144095,5.006124e-03,0.051325,152


In [24]:
fin3

,coef,Prob(F-statistic),R-squared,是否平稳,是否协整
工业增加值:累计同比,-0.012617,2.328634e-01,0.007254,平稳,否
工业增加值:采矿业:累计同比,0.065697,2.433711e-02,0.090412,平稳,否
工业增加值:制造业:累计同比,0.303605,4.493329e-09,0.474073,平稳,否
工业增加值:汽车制造业:累计同比,0.077705,3.103308e-04,0.172922,平稳,是
工业增加值:电力、燃气及水的生产和供应业:累计同比,0.104738,7.150363e-05,0.255243,平稳,否
工业企业:利润总额:累计同比,-0.001867,4.717645e-01,0.003458,平稳,否
工业企业:利润总额:当月同比,0.023665,1.986440e-03,0.146174,平稳,否
工业企业:主营业务收入:累计同比,-0.004001,4.450666e-01,0.003893,平稳,否
工业企业:产成品存货:累计同比,0.020467,7.931679e-03,0.046054,平稳,否
工业企业:资产负债率,-0.144095,5.006124e-03,0.051325,非平稳,否


In [25]:
col = ['工业增加值:累计同比', '工业增加值:汽车制造业:累计同比',
       '工业企业:利润总额:累计同比', '工业企业:利润总额:当月同比',
       '工业企业:主营业务收入:累计同比', '工业企业:产成品存货:累计同比', '工业企业:资产负债率']
csv_for_en(factor3, col, '3.csv', 'month')

# 进出口类指标 4

In [26]:
factor4 = loading_file(3)
factor4

Loading file 4 in 20 ...


,出口金额:当月同比,进口金额:当月同比,贸易差额:当月同比,贸易差额:当月值,美国:进出口金额:累计同比,日本:进出口金额:累计同比,欧盟:进出口金额:累计同比,美国:贸易差额:当月值,欧盟:贸易差额:当月值,日本:贸易差额:当月值
描述,,,,,,,,,,
1995-01-01,88.30,13.70,NaN,26.91,48.0,37.0,17.4,100430.80,31996.60,426.00
1995-02-01,55.30,19.60,NaN,17.71,37.3,37.1,19.7,54938.20,-11953.40,23839.70
1995-03-01,51.70,17.40,NaN,26.33,33.0,33.1,20.4,53835.80,-11327.80,-4622.20
1995-04-01,34.10,9.10,NaN,17.20,30.8,27.4,16.8,33703.90,-4037.70,-3475.70
1995-05-01,35.80,21.70,NaN,13.36,29.4,28.7,15.1,71775.70,-6624.50,-15374.90
1995-06-01,28.70,11.10,NaN,30.80,28.7,27.6,15.2,92589.00,44207.10,7071.80
1995-07-01,18.60,15.60,NaN,11.42,25.7,26.6,16.1,120699.80,-10456.60,-27890.10
1995-08-01,22.70,15.40,NaN,9.44,22.3,26.9,17.3,94260.90,-17532.60,-11513.40
1995-09-01,18.80,22.60,NaN,11.32,20.5,26.9,18.7,81438.00,-17133.10,38007.80


In [27]:
fin4, results4 = auto(3, 'month')
results4

Loading file 4 in 20 ...


,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,出口金额:当月同比,2.247214e-03,4.544218e-01,0.002873,197
1,进口金额:当月同比,1.707735e-03,5.054221e-01,0.002278,197
2,贸易差额:当月同比,9.751714e-06,7.061685e-01,0.000730,197
3,贸易差额:当月值,9.674906e-04,1.263118e-03,0.052044,197
4,美国:进出口金额:累计同比,4.232218e-03,2.578658e-01,0.006560,197
5,日本:进出口金额:累计同比,4.312704e-03,1.868759e-01,0.008917,197
6,欧盟:进出口金额:累计同比,5.725081e-03,5.948951e-02,0.018094,197
7,美国:贸易差额:当月值,4.941679e-07,5.643139e-11,0.198093,197
8,欧盟:贸易差额:当月值,5.704682e-07,8.840917e-07,0.116832,197
9,日本:贸易差额:当月值,-3.556866e-07,2.836016e-01,0.005894,197


In [28]:
fin4

,coef,Prob(F-statistic),R-squared,是否平稳,是否协整
出口金额:当月同比,2.247214e-03,4.544218e-01,0.002873,平稳,否
进口金额:当月同比,1.707735e-03,5.054221e-01,0.002278,平稳,否
贸易差额:当月同比,9.751714e-06,7.061685e-01,0.000730,平稳,是
贸易差额:当月值,9.674906e-04,1.263118e-03,0.052044,平稳,否
美国:进出口金额:累计同比,4.232218e-03,2.578658e-01,0.006560,平稳,否
日本:进出口金额:累计同比,4.312704e-03,1.868759e-01,0.008917,平稳,否
欧盟:进出口金额:累计同比,5.725081e-03,5.948951e-02,0.018094,平稳,否
美国:贸易差额:当月值,4.941679e-07,5.643139e-11,0.198093,平稳,否
欧盟:贸易差额:当月值,5.704682e-07,8.840917e-07,0.116832,平稳,否
日本:贸易差额:当月值,-3.556866e-07,2.836016e-01,0.005894,平稳,否


In [29]:
col = ['出口金额:当月同比', '进口金额:当月同比', '贸易差额:当月同比', '贸易差额:当月值', '美国:进出口金额:累计同比',
       '日本:进出口金额:累计同比', '欧盟:进出口金额:累计同比', '美国:贸易差额:当月值', '欧盟:贸易差额:当月值',
       '日本:贸易差额:当月值']
csv_for_en(factor4, col, '4.csv', 'month')

# 房地产类指标 5

In [30]:
factor5 = loading_file(4)
factor5

Loading file 5 in 20 ...


,房地产开发投资完成额:累计同比,待开发土地面积:累计同比,本年购置土地面积:累计同比,本年土地成交价款:累计同比,土地购置费:累计同比,房屋施工面积:累计同比,房屋新开工面积:累计同比,房屋竣工面积:累计同比,商品房销售面积:累计同比,商品房销售额:累计同比,商品房待售面积:累计同比,商品房待售面积:累计值
描述,,,,,,,,,,,,
1992-02-01,NaN,NaN,NaN,NaN,NaN,39.6,128.1,10.6,18.60,NaN,NaN,NaN
1992-03-01,NaN,NaN,NaN,NaN,NaN,43.6,171.5,1.9,43.60,NaN,NaN,NaN
1992-04-01,NaN,NaN,NaN,NaN,NaN,50.4,148.4,8.0,35.90,NaN,NaN,NaN
1992-05-01,NaN,NaN,NaN,NaN,NaN,47.2,94.0,26.8,27.40,NaN,NaN,NaN
1992-06-01,NaN,NaN,NaN,NaN,NaN,28.7,71.0,10.3,-0.02,NaN,NaN,NaN
1992-07-01,NaN,NaN,NaN,NaN,NaN,29.6,78.3,11.4,9.80,NaN,NaN,NaN
1992-08-01,NaN,NaN,NaN,NaN,NaN,35.4,86.0,25.1,13.30,NaN,NaN,NaN
1992-09-01,NaN,NaN,NaN,NaN,NaN,39.0,95.8,26.2,28.70,NaN,NaN,NaN
1992-10-01,NaN,NaN,NaN,NaN,NaN,39.0,93.7,27.2,27.00,NaN,NaN,NaN


In [31]:
fin5, results5 = auto(4, 'month')
results5

Loading file 5 in 20 ...


,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,房地产开发投资完成额:累计同比,-0.000408,9.357463e-01,0.000037,180
1,待开发土地面积:累计同比,0.023095,5.517614e-09,0.256873,117
2,本年购置土地面积:累计同比,0.000107,9.628157e-01,0.000012,180
3,本年土地成交价款:累计同比,0.004184,4.192190e-02,0.028233,147
4,土地购置费:累计同比,0.000977,6.838556e-01,0.000934,180
5,房屋施工面积:累计同比,-0.007628,1.935052e-01,0.009480,180
6,房屋新开工面积:累计同比,-0.006456,2.797637e-02,0.026841,180
7,房屋竣工面积:累计同比,-0.022146,2.960114e-08,0.158979,180
8,商品房销售面积:累计同比,-0.015179,4.092741e-06,0.112681,180
9,商品房销售额:累计同比,-0.010902,3.414176e-08,0.157666,180


In [32]:
fin5

,coef,Prob(F-statistic),R-squared,是否平稳,是否协整
房地产开发投资完成额:累计同比,-0.000408,9.357463e-01,0.000037,平稳,否
待开发土地面积:累计同比,0.023095,5.517614e-09,0.256873,平稳,否
本年购置土地面积:累计同比,0.000107,9.628157e-01,0.000012,平稳,是
本年土地成交价款:累计同比,0.004184,4.192190e-02,0.028233,平稳,否
土地购置费:累计同比,0.000977,6.838556e-01,0.000934,平稳,否
房屋施工面积:累计同比,-0.007628,1.935052e-01,0.009480,平稳,否
房屋新开工面积:累计同比,-0.006456,2.797637e-02,0.026841,平稳,否
房屋竣工面积:累计同比,-0.022146,2.960114e-08,0.158979,平稳,否
商品房销售面积:累计同比,-0.015179,4.092741e-06,0.112681,平稳,否
商品房销售额:累计同比,-0.010902,3.414176e-08,0.157666,平稳,是


In [33]:
col = ['房地产开发投资完成额:累计同比', '待开发土地面积:累计同比', '本年购置土地面积:累计同比', '本年土地成交价款:累计同比',
       '土地购置费:累计同比', '房屋施工面积:累计同比', '房屋新开工面积:累计同比', '房屋竣工面积:累计同比',
       '商品房销售面积:累计同比', '商品房销售额:累计同比', '商品房待售面积:累计同比', '商品房待售面积:累计值']
csv_for_en(factor5, col, '5.csv', 'month')

# 消费类指标 6

In [34]:
factor6 = loading_file(5)
factor6

Loading file 6 in 20 ...


,社会消费品零售总额:累计同比,社会消费品零售总额:实际累计同比,社会消费品零售总额:城镇:累计同比,社会消费品零售总额:乡村:累计同比,消费者信心指数(月)
描述,,,,,
1953-12-01,25.72,NaN,NaN,NaN,NaN
1954-12-01,9.51,NaN,NaN,NaN,NaN
1955-12-01,2.91,NaN,NaN,NaN,NaN
1956-12-01,17.54,NaN,NaN,NaN,NaN
1957-12-01,2.86,NaN,NaN,NaN,NaN
1958-12-01,15.56,NaN,NaN,NaN,NaN
1959-12-01,16.42,NaN,NaN,NaN,NaN
1960-12-01,9.23,NaN,NaN,NaN,NaN
1961-12-01,-12.80,NaN,NaN,NaN,NaN


In [35]:
fin6, results6 = auto(5, 'month')
results6

Loading file 6 in 20 ...


,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,社会消费品零售总额:累计同比,0.018837,2.218120e-01,0.007887,191
1,社会消费品零售总额:实际累计同比,-0.146822,8.012569e-10,0.192578,179
2,社会消费品零售总额:城镇:累计同比,-0.037587,7.567609e-02,0.033896,94
3,社会消费品零售总额:乡村:累计同比,0.023162,4.960091e-01,0.005052,94
4,消费者信心指数(月),0.010057,2.945464e-01,0.005633,197


In [36]:
fin6

,coef,Prob(F-statistic),R-squared,是否平稳,是否协整
社会消费品零售总额:累计同比,0.018837,2.218120e-01,0.007887,平稳,否
社会消费品零售总额:实际累计同比,-0.146822,8.012569e-10,0.192578,非平稳,否
社会消费品零售总额:城镇:累计同比,-0.037587,7.567609e-02,0.033896,平稳,否
社会消费品零售总额:乡村:累计同比,0.023162,4.960091e-01,0.005052,非平稳,否
消费者信心指数(月),0.010057,2.945464e-01,0.005633,平稳,否


In [37]:
col = ['社会消费品零售总额:累计同比', '社会消费品零售总额:实际累计同比', '社会消费品零售总额:城镇:累计同比',
       '社会消费品零售总额:乡村:累计同比', '消费者信心指数(月)']
csv_for_en(factor6, col, '6.csv', 'month')

# 价格指数类指标 7

In [38]:
factor7 = loading_file(6)
factor7

Loading file 7 in 20 ...


,CPI:当月同比,CPI:食品:当月同比,CPI:非食品:当月同比,CPI:不包括食品和能源(核心CPI):当月同比,CPI:不包括鲜菜和鲜果:当月同比,CPI:消费品:当月同比,CPI:服务:当月同比,CPI:食品烟酒:畜肉类:猪肉:当月同比,PPI:全部工业品:当月同比,PPI:生产资料:当月同比,PPI:生产资料:采掘工业:当月同比,PPI:生产资料:原材料工业:当月同比,PPI:生产资料:加工工业:当月同比,PPI:生活资料:当月同比,PPI:生活资料:食品类:当月同比,PPI:生活资料:衣着类:当月同比,PPI:生活资料:一般日用品类:当月同比,PPI:生活资料:耐用消费品类:当月同比
描述,,,,,,,,,,,,,,,,,,
1990-01-01,4.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-01,4.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-03-01,3.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-04-01,3.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-05-01,2.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-06-01,1.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-07-01,1.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-08-01,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-09-01,2.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
fin7, results7 = auto(6, 'month')
results7

Loading file 7 in 20 ...


,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,CPI:当月同比,0.203539,9.868996e-19,0.329263,198
1,CPI:食品:当月同比,0.052568,2.766146e-08,0.146040,198
2,CPI:非食品:当月同比,0.474421,1.661048e-28,0.465966,198
3,CPI:不包括食品和能源(核心CPI):当月同比,0.677113,9.209643e-03,0.101284,66
4,CPI:不包括鲜菜和鲜果:当月同比,0.214194,1.888933e-15,0.328709,161
5,CPI:消费品:当月同比,0.142471,3.617317e-14,0.258843,194
6,CPI:服务:当月同比,0.363741,1.585380e-17,0.314171,195
7,CPI:食品烟酒:畜肉类:猪肉:当月同比,0.011513,6.203415e-06,0.120189,162
8,PPI:全部工业品:当月同比,0.051068,9.983360e-06,0.094997,198
9,PPI:生产资料:当月同比,0.036620,8.466929e-05,0.075990,198


In [72]:
coin

,指标变化率的时间序列,P值,是否协整
0,CPI:当月同比,0.316402,否
1,CPI:食品:当月同比,0.360230,否
2,CPI:非食品:当月同比,0.784337,否
3,CPI:不包括食品和能源(核心CPI):当月同比,0.797208,否
4,CPI:不包括鲜菜和鲜果:当月同比,0.759168,否
5,CPI:消费品:当月同比,0.248982,否
6,CPI:服务:当月同比,0.835656,否
7,CPI:食品烟酒:畜肉类:猪肉:当月同比,0.630291,否
8,PPI:全部工业品:当月同比,0.295048,否
9,PPI:生产资料:当月同比,0.303403,否


In [73]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,CPI:当月同比,-0.127096,5.205150e-01,0.010397,42
1,CPI:食品:当月同比,-0.118100,2.297647e-05,0.364630,42
2,CPI:非食品:当月同比,0.599383,2.283942e-06,0.431748,42
3,CPI:不包括食品和能源(核心CPI):当月同比,1.166029,2.406389e-06,0.430305,42
4,CPI:不包括鲜菜和鲜果:当月同比,0.060506,8.168660e-01,0.001357,42
5,CPI:消费品:当月同比,-0.339686,7.886004e-03,0.163614,42
6,CPI:服务:当月同比,0.731000,7.865930e-06,0.396655,42
7,CPI:食品烟酒:畜肉类:猪肉:当月同比,-0.029700,3.492738e-10,0.630567,42
8,PPI:全部工业品:当月同比,0.071844,2.741215e-06,0.426691,42
9,PPI:生产资料:当月同比,0.054888,2.688839e-06,0.427228,42


In [75]:
col = ['CPI:当月同比', 'CPI:食品:当月同比', 'CPI:非食品:当月同比', 'CPI:不包括食品和能源(核心CPI):当月同比',
       'CPI:不包括鲜菜和鲜果:当月同比', 'CPI:消费品:当月同比', 'CPI:服务:当月同比',
       'CPI:食品烟酒:畜肉类:猪肉:当月同比', 'PPI:全部工业品:当月同比', 'PPI:生产资料:当月同比',
       'PPI:生产资料:采掘工业:当月同比', 'PPI:生产资料:原材料工业:当月同比', 'PPI:生产资料:加工工业:当月同比',
       'PPI:生活资料:当月同比', 'PPI:生活资料:食品类:当月同比', 'PPI:生活资料:衣着类:当月同比',
       'PPI:生活资料:一般日用品类:当月同比', 'PPI:生活资料:耐用消费品类:当月同比']
csv_for_en(factor7, col, '7.csv', 'month')

# 发改委价格 8

In [76]:
factor8 = loading_file(7)
factor8

Loading file 8 in 20 ...


,36个城市平均零售价:晚籼米,36个城市平均零售价:粳米,36个城市平均零售价:面粉:富强粉,36个城市平均零售价:面粉:标准粉,36个城市平均零售价:花生油:桶装,36个城市平均零售价:菜籽油:桶装,36个城市平均零售价:豆油:桶装,36个城市平均零售价:猪肉,36个城市平均零售价:牛肉,36个城市平均零售价:羊肉,...,36个大中城市平均零售价:尖椒,36个大中城市平均零售价:圆白菜,36个大中城市平均零售价:豆角,36个大中城市平均零售价:蒜苔,36个大中城市平均零售价:韭菜,36个大中城市平均零售价:青椒,36个大中城市平均零售价:黄瓜,36个大中城市平均零售价:西红柿,36个大中城市平均零售价:油菜,36个大中城市平均零售价:茄子
描述,,,,,,,,,,,,,,,,,,,,,
2015-06-25,2.50,2.77,2.57,2.27,130.15,62.64,49.55,14.58,34.49,33.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-26,2.50,2.78,2.57,2.27,130.14,62.64,49.55,14.60,34.49,33.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-29,2.50,2.78,2.57,2.27,130.14,62.67,49.53,14.66,34.46,33.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-06-30,2.50,2.78,2.57,2.27,130.11,62.67,49.53,14.69,34.44,33.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-01,2.50,2.78,2.57,2.27,130.15,62.67,49.57,14.72,34.47,33.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-02,2.50,2.77,2.57,2.27,130.16,62.73,49.60,14.75,34.49,33.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-03,2.50,2.77,2.57,2.27,130.20,62.75,49.64,14.82,34.59,33.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-06,2.50,2.78,2.57,2.27,130.22,62.75,49.59,14.95,34.67,33.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-07,2.50,2.78,2.57,2.27,130.18,62.75,49.61,15.02,34.53,33.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
coin = cointest(factor8, 'day')
results = reg(factor8, 'day')
coin

,指标变化率的时间序列,P值,是否协整
0,36个城市平均零售价:晚籼米,0.674676,否
1,36个城市平均零售价:粳米,0.203239,否
2,36个城市平均零售价:面粉:富强粉,0.366116,否
3,36个城市平均零售价:面粉:标准粉,0.788228,否
4,36个城市平均零售价:花生油:桶装,0.032816,是
5,36个城市平均零售价:菜籽油:桶装,0.285814,否
6,36个城市平均零售价:豆油:桶装,0.725170,否
7,36个城市平均零售价:猪肉,0.749292,否
8,36个城市平均零售价:牛肉,0.737974,否
9,36个城市平均零售价:羊肉,0.890432,否


In [79]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,36个城市平均零售价:晚籼米,13.787296,3.206424e-90,0.419239,749
1,36个城市平均零售价:粳米,25.460198,6.625639e-97,0.442646,749
2,36个城市平均零售价:面粉:富强粉,7.460179,5.685430e-253,0.786884,749
3,36个城市平均零售价:面粉:标准粉,10.373149,4.460071e-187,0.680143,749
4,36个城市平均零售价:花生油:桶装,0.568003,2.973839e-213,0.727785,749
5,36个城市平均零售价:菜籽油:桶装,0.800663,2.152991e-36,0.191535,749
6,36个城市平均零售价:豆油:桶装,-0.816595,4.133350e-81,0.385756,749
7,36个城市平均零售价:猪肉,-0.310714,2.763716e-58,0.293204,749
8,36个城市平均零售价:牛肉,0.389457,8.080750e-16,0.083206,749
9,36个城市平均零售价:羊肉,0.021284,2.649164e-01,0.001664,749


In [81]:
col = ['36个城市平均零售价:晚籼米', '36个城市平均零售价:粳米', '36个城市平均零售价:面粉:富强粉',
       '36个城市平均零售价:面粉:标准粉', '36个城市平均零售价:花生油:桶装', '36个城市平均零售价:菜籽油:桶装',
       '36个城市平均零售价:豆油:桶装', '36个城市平均零售价:猪肉', '36个城市平均零售价:牛肉', '36个城市平均零售价:羊肉',
       '36个城市平均零售价:鸡肉', '36个城市平均零售价:鸡蛋', '36个城市平均零售价:草鱼', '36个城市平均零售价:鲢鱼',
       '36个大中城市平均零售价:带鱼', '36个大中城市平均零售价:芹菜', '36个大中城市平均零售价:大白菜',
       '36个大中城市平均零售价:萝卜', '36个大中城市平均零售价:土豆', '36个大中城市平均零售价:胡萝卜',
       '36个大中城市平均零售价:尖椒', '36个大中城市平均零售价:圆白菜', '36个大中城市平均零售价:豆角',
       '36个大中城市平均零售价:蒜苔', '36个大中城市平均零售价:韭菜', '36个大中城市平均零售价:青椒',
       '36个大中城市平均零售价:黄瓜', '36个大中城市平均零售价:西红柿', '36个大中城市平均零售价:油菜',
       '36个大中城市平均零售价:茄子']
csv_for_en(factor8, col, '8.csv', 'month')

# 统计局生产资料 9

In [22]:
factor9 = loading_file(8)
factor9

Loading file 9 in 22 ...


,市场价:螺纹钢:HRB400 Φ16-25mm:全国,市场价:线材:HPB300 Φ6.5mm:全国,市场价:普通中板:Q235 20mm:全国,市场价:热轧普通薄板:Q235 3mm:全国,市场价:无缝钢管:20# 219*6:全国,市场价:角钢:5#:全国,市场价:电解铜:1#:全国,市场价:铝锭:A00:全国,市场价:铅锭:1#:全国,市场价:锌锭:0#:全国,...,市场价:生猪:外三元:全国,市场价:大豆:黄豆:全国,市场价:豆粕:粗蛋白含量≥43%:全国,市场价:花生:油料花生米:全国,市场价:尿素(小颗料):全国,市场价:硫酸钾复合肥:全国,市场价:草甘膦(95%原药):全国,市场价:人造板:1220*2440*15mm:全国,市场价:纸浆:漂白化学浆:全国,市场价:瓦楞纸:高强:全国
描述,,,,,,,,,,,,,,,,,,,,,
2013-12-31,3503.6,3531.4,3533.4,3589.4,4401.7,3593.9,51904.4,14136.3,14140.0,15237.7,...,15.3,4736.4,3943.8,6671.4,1751.7,2960.0,35375.0,54.0,4793.7,2792.0
2014-01-10,3456.6,3488.0,3517.9,3585.6,4399.1,3589.4,52113.1,14080.7,14079.3,15127.0,...,14.2,4739.6,3871.1,6438.5,1737.2,2960.0,35375.0,53.8,4778.7,2792.0
2014-01-20,3409.0,3427.5,3497.8,3577.3,4400.0,3580.1,52001.1,13739.5,14047.9,15174.7,...,12.8,4733.3,3823.8,6309.7,1725.3,2918.0,35166.7,53.7,4774.2,2792.0
2014-01-31,3393.9,3396.9,3493.5,3569.4,4390.5,3564.6,51241.1,13565.9,14007.9,15135.9,...,12.3,4730.0,3802.0,6266.7,1736.9,2933.0,34125.0,53.7,4780.0,2792.0
2014-02-10,3394.8,3391.0,3494.8,3563.8,4390.5,3563.9,50965.5,13261.0,13990.0,15069.5,...,12.5,4715.8,3825.0,6266.7,1735.8,2948.0,34125.0,53.7,4797.5,2792.0
2014-02-20,3357.6,3349.3,3496.2,3547.2,4391.1,3554.7,50657.9,13166.3,13921.0,15075.5,...,12.1,4726.3,3830.2,6289.6,1730.4,2963.0,34125.0,53.7,4832.9,2797.0
2014-02-28,3316.7,3306.3,3496.6,3528.0,4387.3,3546.1,49598.5,13043.0,13835.1,14999.2,...,11.7,4729.4,3763.3,6123.6,1711.5,2966.0,34125.0,53.7,4849.6,2802.0
2014-03-10,3296.4,3303.0,3486.9,3499.6,4367.9,3534.3,48489.5,12852.8,13722.9,14977.0,...,11.3,4714.7,3631.3,5960.0,1688.5,2966.0,34625.0,53.7,4825.0,2794.0
2014-03-20,3260.2,3274.0,3465.1,3457.7,4326.2,3515.7,44994.6,12629.0,13639.4,14745.4,...,11.1,4674.0,3501.7,5934.4,1654.3,2966.0,35312.5,53.7,4807.3,2768.5


In [23]:
coin = cointest(factor9, 'special day')
results = reg(factor9, 'special day')
coin

,指标变化率的时间序列,P值,是否协整
0,市场价:螺纹钢:HRB400 Φ16-25mm:全国,0.938661,否
1,市场价:线材:HPB300 Φ6.5mm:全国,0.952959,否
2,市场价:普通中板:Q235 20mm:全国,0.925920,否
3,市场价:热轧普通薄板:Q235 3mm:全国,0.891162,否
4,市场价:无缝钢管:20# 219*6:全国,0.894085,否
5,市场价:角钢:5#:全国,0.954862,否
6,市场价:电解铜:1#:全国,0.321967,否
7,市场价:铝锭:A00:全国,0.336353,否
8,市场价:铅锭:1#:全国,0.950762,否
9,市场价:锌锭:0#:全国,0.852700,否


In [89]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,市场价:螺纹钢:HRB400 Φ16-25mm:全国,0.000530,2.121390e-30,0.663537,123
1,市场价:线材:HPB300 Φ6.5mm:全国,0.000521,4.398845e-30,0.659474,123
2,市场价:普通中板:Q235 20mm:全国,0.000502,5.179225e-24,0.571456,123
3,市场价:热轧普通薄板:Q235 3mm:全国,0.000487,7.585844e-22,0.534883,123
4,市场价:无缝钢管:20# 219*6:全国,0.000470,2.533733e-31,0.675105,123
5,市场价:角钢:5#:全国,0.000561,8.103360e-32,0.681146,123
6,市场价:电解铜:1#:全国,0.000072,2.102924e-29,0.650591,123
7,市场价:铝锭:A00:全国,0.000250,1.178155e-16,0.434226,123
8,市场价:铅锭:1#:全国,0.000125,5.588771e-16,0.419636,123
9,市场价:锌锭:0#:全国,0.000092,4.876116e-25,0.587777,123


In [92]:
col = ['市场价:螺纹钢:HRB400 Φ16-25mm:全国', '市场价:线材:HPB300 Φ6.5mm:全国',
       '市场价:普通中板:Q235 20mm:全国', '市场价:热轧普通薄板:Q235 3mm:全国',
       '市场价:无缝钢管:20# 219*6:全国', '市场价:角钢:5#:全国', '市场价:电解铜:1#:全国',
       '市场价:铝锭:A00:全国', '市场价:铅锭:1#:全国', '市场价:锌锭:0#:全国', '市场价:硫酸(98%):全国',
       '市场价:液碱(32%):全国', '市场价:甲醇(优等品):全国', '市场价:石油苯(工业级):全国',
       '市场价:苯乙烯(一级品):全国', '市场价:LLDPE(7042):全国', '市场价:聚丙烯(T30S):全国',
       '市场价:聚氯乙烯(SG5):全国', '市场价:顺丁橡胶(BR9000):全国', '市场价:涤纶长丝(FDY150D/96F):全国',
       '市场价:液化天然气LNG:全国', '市场价:液化气LPG:全国', '市场价:汽油(97#):全国', '市场价:汽油(93#):全国',
       '市场价:柴油(0#):全国', '市场价:石蜡(58#半):全国', '市场价:无烟煤(2号洗中块):全国',
       '市场价:普通混煤(Q4500):全国', '市场价:山西大混(Q5000):全国', '市场价:山西优混(Q5500):全国',
       '市场价:大同混煤(Q5800):全国', '市场价:1/3焦煤:全国', '市场价:二级冶金焦:全国',
       '市场价:复合硅酸盐水泥:P.C32.5 袋装:全国', '市场价:普通硅酸盐水泥:P.O42.5 散装:全国',
       '市场价:浮法平板玻璃:4.8/5mm:全国', '市场价:稻米:粳稻米:全国', '市场价:小麦:国标三等:全国',
       '市场价:玉米:黄玉米二等:全国', '市场价:棉花:皮棉 白棉三级:全国', '市场价:生猪:外三元:全国', '市场价:大豆:黄豆:全国',
       '市场价:豆粕:粗蛋白含量≥43%:全国', '市场价:花生:油料花生米:全国', '市场价:尿素(小颗料):全国',
       '市场价:硫酸钾复合肥:全国', '市场价:草甘膦(95%原药):全国', '市场价:人造板:1220*2440*15mm:全国',
       '市场价:纸浆:漂白化学浆:全国', '市场价:瓦楞纸:高强:全国']
csv_for_en(factor9, col, '9.csv', 'special day')

# 统计局食品类 10

In [25]:
factor10 = loading_file(9)
factor10

Loading file 10 in 22 ...


,50个城市平均价:面粉:标准粉,50个城市平均价:面粉:富强粉,50个城市平均价:大米,50个城市平均价:豆制品:豆腐,50个城市平均价:花生油:压榨一级,50个城市平均价:大豆油:5L桶装,50个城市平均价:菜籽油:一级散装,50个城市平均价:猪肉:猪肉后臀尖(后腿肉),50个城市平均价:猪肉:五花肉,50个城市平均价:牛肉:腿肉,...,50个城市平均价:带鱼,50个城市平均价:大白菜,50个城市平均价:油菜,50个城市平均价:芹菜,50个城市平均价:黄瓜,50个城市平均价:西红柿,50个城市平均价:豆角,50个城市平均价:土豆,50个城市平均价:苹果:富士,50个城市平均价:香蕉:国产
描述,,,,,,,,,,,,,,,,,,,,,
2009-02-20,3.08,3.76,3.69,3.40,20.72,10.09,10.34,21.69,20.77,34.69,...,16.77,1.47,3.96,3.88,5.86,4.96,8.31,2.58,6.66,4.26
2009-02-28,3.09,3.77,3.72,3.41,20.64,10.06,10.34,21.15,20.23,34.43,...,16.50,1.52,4.14,3.92,5.45,4.90,8.19,2.61,6.56,4.10
2009-03-10,3.12,3.77,3.76,3.41,20.29,10.12,10.19,20.49,19.61,34.50,...,16.39,1.81,4.73,4.36,5.62,5.10,9.11,2.70,6.63,4.13
2009-03-20,3.13,3.78,3.79,3.42,20.20,9.94,10.09,19.89,19.16,34.43,...,16.32,2.06,4.34,4.43,5.49,5.13,9.19,2.75,6.66,4.30
2009-03-30,3.15,3.79,3.79,3.40,20.01,9.78,9.97,19.46,18.65,34.36,...,16.40,2.25,4.03,4.23,5.12,4.87,8.33,2.76,6.69,4.56
2009-04-10,3.17,3.81,3.80,3.40,19.72,9.64,9.96,18.79,17.87,34.20,...,16.40,2.69,3.67,4.27,4.76,4.98,8.53,2.89,6.90,4.83
2009-04-20,3.15,3.81,3.80,3.39,19.97,9.50,9.90,18.41,17.46,34.20,...,16.36,2.74,3.31,4.08,4.29,5.06,7.45,3.09,7.17,5.27
2009-04-30,3.16,3.83,3.81,3.39,19.87,9.42,9.85,17.99,17.03,34.12,...,16.53,2.70,3.33,4.07,4.38,4.87,7.55,3.16,7.49,5.73
2009-05-10,3.17,3.82,3.86,3.39,19.86,9.43,9.91,17.50,16.55,34.03,...,16.58,2.61,3.23,4.12,3.96,4.80,6.85,3.18,7.78,5.96


In [26]:
coin = cointest(factor10, 'special day')
results = reg(factor10, 'special day')
coin

,指标变化率的时间序列,P值,是否协整
0,50个城市平均价:面粉:标准粉,0.927509,否
1,50个城市平均价:面粉:富强粉,0.857843,否
2,50个城市平均价:大米,0.725702,否
3,50个城市平均价:豆制品:豆腐,0.926747,否
4,50个城市平均价:花生油:压榨一级,0.788011,否
5,50个城市平均价:大豆油:5L桶装,0.936824,否
6,50个城市平均价:菜籽油:一级散装,0.690249,否
7,50个城市平均价:猪肉:猪肉后臀尖(后腿肉),0.542949,否
8,50个城市平均价:猪肉:五花肉,0.640061,否
9,50个城市平均价:牛肉:腿肉,0.885486,否


In [27]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,50个城市平均价:面粉:标准粉,0.732084,4.204646e-30,0.339321,316
1,50个城市平均价:面粉:富强粉,0.549010,8.435328e-27,0.306753,316
2,50个城市平均价:大米,0.650064,1.540487e-38,0.415837,316
3,50个城市平均价:豆制品:豆腐,0.785166,1.078556e-22,0.264059,316
4,50个城市平均价:花生油:压榨一级,0.126811,2.302513e-31,0.351361,316
5,50个城市平均价:大豆油:5L桶装,0.341276,1.390330e-15,0.184083,316
6,50个城市平均价:菜籽油:一级散装,0.421405,6.464137e-51,0.512489,316
7,50个城市平均价:猪肉:猪肉后臀尖(后腿肉),0.089955,4.977669e-17,0.200989,316
8,50个城市平均价:猪肉:五花肉,0.081402,6.579681e-18,0.211102,316
9,50个城市平均价:牛肉:腿肉,0.028631,4.119195e-23,0.268519,316


In [97]:
col = ['50个城市平均价:面粉:标准粉', '50个城市平均价:面粉:富强粉', '50个城市平均价:大米', '50个城市平均价:豆制品:豆腐',
       '50个城市平均价:花生油:压榨一级', '50个城市平均价:大豆油:5L桶装', '50个城市平均价:菜籽油:一级散装',
       '50个城市平均价:猪肉:猪肉后臀尖(后腿肉)', '50个城市平均价:猪肉:五花肉', '50个城市平均价:牛肉:腿肉',
       '50个城市平均价:羊肉:腿肉', '50个城市平均价:鸡:白条鸡', '50个城市平均价:鸡:鸡胸肉', '50个城市平均价:鸭',
       '50个城市平均价:鸡蛋', '50个城市平均价:活鲤鱼', '50个城市平均价:活草鱼', '50个城市平均价:带鱼',
       '50个城市平均价:大白菜', '50个城市平均价:油菜', '50个城市平均价:芹菜', '50个城市平均价:黄瓜',
       '50个城市平均价:西红柿', '50个城市平均价:豆角', '50个城市平均价:土豆', '50个城市平均价:苹果:富士',
       '50个城市平均价:香蕉:国产']
csv_for_en(factor10, col, '10.csv', 'special day')

# 资金利率类 11

In [28]:
factor11 = loading_file(10)
factor11

Loading file 11 in 22 ...


,银行间质押式回购加权利率:1天,银行间质押式回购加权利率:7天,银行间质押式回购加权利率:14天,银行间质押式回购加权利率:21天,银行间质押式回购加权利率:1个月,银行间质押式回购加权利率:2个月,银行间质押式回购加权利率:3个月,银行间质押式回购加权利率:4个月,银行间质押式回购加权利率:6个月,银行间质押式回购加权利率:9个月,...,SHIBOR:1年,GC001:加权平均,GC002:加权平均,GC003:加权平均,GC004:加权平均,GC007:加权平均,GC014:加权平均,GC028:加权平均,GC091:加权平均,GC182:加权平均
描述,,,,,,,,,,,,,,,,,,,,,
1999-01-04,NaN,4.0200,NaN,NaN,NaN,NaN,4.9500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-05,NaN,4.0500,4.0500,NaN,NaN,NaN,4.8000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-06,NaN,4.0500,4.0300,NaN,NaN,NaN,4.9800,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-07,NaN,4.0500,NaN,NaN,4.7190,NaN,4.8520,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-08,NaN,4.0500,NaN,NaN,4.4100,NaN,NaN,4.8000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-11,NaN,NaN,4.0500,4.5000,4.2000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-12,NaN,4.0500,4.0500,4.1320,NaN,NaN,4.5900,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-13,NaN,4.1360,4.0570,NaN,4.2000,NaN,4.9500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-14,NaN,4.0500,NaN,NaN,4.2300,NaN,4.5750,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
coin = cointest(factor11, 'day')
results = reg(factor11, 'day')
coin

,指标变化率的时间序列,P值,是否协整
0,银行间质押式回购加权利率:1天,3.846384e-08,是
1,银行间质押式回购加权利率:7天,9.484973e-08,是
2,银行间质押式回购加权利率:14天,1.616013e-06,是
3,银行间质押式回购加权利率:21天,4.245237e-06,是
4,银行间质押式回购加权利率:1个月,3.033179e-05,是
5,银行间质押式回购加权利率:2个月,1.701231e-05,是
6,银行间质押式回购加权利率:3个月,9.588000e-04,是
7,银行间质押式回购加权利率:4个月,1.482164e-02,是
8,银行间质押式回购加权利率:6个月,2.664020e-02,是
9,银行间质押式回购加权利率:9个月,5.227342e-03,是


In [30]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,银行间质押式回购加权利率:1天,0.522058,0.000000e+00,0.423513,3824
1,银行间质押式回购加权利率:7天,0.455895,0.000000e+00,0.480469,4121
2,银行间质押式回购加权利率:14天,0.417918,0.000000e+00,0.536816,4120
3,银行间质押式回购加权利率:21天,0.395429,0.000000e+00,0.557046,3940
4,银行间质押式回购加权利率:1个月,0.409017,0.000000e+00,0.600859,4004
5,银行间质押式回购加权利率:2个月,0.409312,0.000000e+00,0.627348,3410
6,银行间质押式回购加权利率:3个月,0.436845,0.000000e+00,0.670070,2918
7,银行间质押式回购加权利率:4个月,0.458754,0.000000e+00,0.675582,1424
8,银行间质押式回购加权利率:6个月,0.468082,0.000000e+00,0.697146,1568
9,银行间质押式回购加权利率:9个月,0.513385,2.999872e-144,0.712576,527


In [110]:
col = ['银行间质押式回购加权利率:1天', '银行间质押式回购加权利率:7天', '银行间质押式回购加权利率:14天',
       '银行间质押式回购加权利率:21天', '银行间质押式回购加权利率:1个月', '银行间质押式回购加权利率:2个月',
       '银行间质押式回购加权利率:3个月', '银行间质押式回购加权利率:4个月', '银行间质押式回购加权利率:6个月',
       '银行间质押式回购加权利率:9个月', '银行间质押式回购加权利率:1年', '存款类机构质押式回购加权利率:1天',
       '存款类机构质押式回购加权利率:7天', '存款类机构质押式回购加权利率:14天', '存款类机构质押式回购加权利率:21天',
       '存款类机构质押式回购加权利率:1个月', '存款类机构质押式回购加权利率:2个月', '存款类机构质押式回购加权利率:3个月',
       '存款类机构质押式回购加权利率:4个月', '存款类机构质押式回购加权利率:6个月', '存款类机构质押式回购加权利率:9个月',
       '存款类机构质押式回购加权利率:1年', 'SHIBOR:隔夜', 'SHIBOR:1周', 'SHIBOR:2周',
       'SHIBOR:1个月', 'SHIBOR:3个月', 'SHIBOR:6个月', 'SHIBOR:9个月', 'SHIBOR:1年',
       'GC001:加权平均', 'GC002:加权平均', 'GC003:加权平均', 'GC004:加权平均', 'GC007:加权平均',
       'GC014:加权平均', 'GC028:加权平均', 'GC091:加权平均', 'GC182:加权平均']
csv_for_en(factor11, col, '11.csv', 'special day')

# 广义流动性类 12

In [36]:
factor12 = loading_file(11)
factor12

Loading file 12 in 22 ...


,M1:同比,M2:同比,社会融资规模:当月值,社会融资规模:累计值,社会融资规模:新增人民币贷款:当月值,社会融资规模:新增人民币贷款:累计值,金融机构:短期贷款余额,金融机构:中长期贷款余额,金融机构:各项贷款余额,金融机构:各项贷款余额:同比,...,金融机构:新增人民币贷款:短期贷款及票据融资:当月值,金融机构:新增人民币贷款:票据融资:当月值,金融机构:新增人民币贷款:中长期:当月值,金融机构:新增人民币贷款:居民户:当月值,金融机构:新增人民币贷款:居民户:短期:当月值,金融机构:新增人民币贷款:居民户:中长期:当月值,金融机构:新增人民币贷款:非金融性公司及其他部门:当月值,金融机构:新增人民币贷款:非金融性公司:短期:当月值,金融机构:新增人民币贷款:非金融性公司:中长期:当月值,金融机构:新增人民币贷款:非银行业金融机构
描述,,,,,,,,,,,,,,,,,,,,,
1978-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1890.42,11.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2082.47,10.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2478.08,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2853.29,15.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3162.70,10.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3566.56,12.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4746.80,33.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6198.38,30.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-12-01,26.7,29.3,NaN,NaN,NaN,NaN,NaN,NaN,8142.72,31.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
adf = ADFtest(factor12)
coin = cointest(factor12, 'month')
results = reg(factor12, 'month')
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,M1:同比,-5.978806,-3.455175,-2.872468,平稳
1,M2:同比,-5.086972,-3.455558,-2.872636,平稳
2,社会融资规模:当月值,-4.697922,-3.466800,-2.877555,平稳
3,社会融资规模:累计值,-3.518631,-3.466800,-2.877555,平稳
4,社会融资规模:新增人民币贷款:当月值,-4.861916,-3.467211,-2.877735,平稳
5,社会融资规模:新增人民币贷款:累计值,-4.399554,-3.467005,-2.877644,平稳
6,金融机构:短期贷款余额,-1.363849,-3.454267,-2.872070,非平稳
7,金融机构:中长期贷款余额,-0.874661,-3.454180,-2.872031,非平稳
8,金融机构:各项贷款余额,0.454043,-3.451082,-2.870672,非平稳
9,金融机构:各项贷款余额:同比,-6.283666,-3.451484,-2.870849,平稳


In [8]:
coin

,指标变化率的时间序列,P值,是否协整
0,M1:同比,7.010700e-01,否
1,M2:同比,6.579903e-01,否
2,社会融资规模:当月值,4.346531e-02,是
3,社会融资规模:累计值,8.472625e-01,否
4,社会融资规模:新增人民币贷款:当月值,3.724645e-08,是
5,社会融资规模:新增人民币贷款:累计值,8.330886e-01,否
6,金融机构:短期贷款余额,9.559783e-01,否
7,金融机构:中长期贷款余额,8.670629e-01,否
8,金融机构:各项贷款余额,8.872115e-01,否
9,金融机构:各项贷款余额:同比,5.333854e-01,否


In [38]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,M1:同比,-3.980053e-02,2.072531e-08,0.149188,197
1,M2:同比,-9.597520e-02,2.230904e-18,0.325073,197
2,社会融资规模:当月值,1.968228e-05,1.043392e-02,0.033161,197
3,社会融资规模:累计值,4.923861e-06,2.930470e-06,0.106309,197
4,社会融资规模:新增人民币贷款:当月值,2.124781e-05,7.116893e-02,0.016600,197
5,社会融资规模:新增人民币贷款:累计值,5.115735e-06,1.999987e-03,0.047911,197
6,金融机构:短期贷款余额,2.925965e-06,1.749480e-10,0.188920,197
7,金融机构:中长期贷款余额,1.435333e-06,1.705836e-09,0.170175,197
8,金融机构:各项贷款余额,8.761389e-07,3.692141e-09,0.163734,197
9,金融机构:各项贷款余额:同比,-7.226281e-02,2.568694e-12,0.222660,197


In [11]:
col = ['M1:同比', 'M2:同比', '社会融资规模:当月值', '社会融资规模:累计值', '社会融资规模:新增人民币贷款:当月值',
       '社会融资规模:新增人民币贷款:累计值', '金融机构:短期贷款余额', '金融机构:中长期贷款余额', '金融机构:各项贷款余额',
       '金融机构:各项贷款余额:同比', '金融机构:各项存款余额', '金融机构:各项存款余额:同比',
       '金融机构:新增人民币贷款:短期贷款及票据融资:当月值', '金融机构:新增人民币贷款:票据融资:当月值',
       '金融机构:新增人民币贷款:中长期:当月值', '金融机构:新增人民币贷款:居民户:当月值',
       '金融机构:新增人民币贷款:居民户:短期:当月值', '金融机构:新增人民币贷款:居民户:中长期:当月值',
       '金融机构:新增人民币贷款:非金融性公司及其他部门:当月值', '金融机构:新增人民币贷款:非金融性公司:短期:当月值',
       '金融机构:新增人民币贷款:非金融性公司:中长期:当月值']
csv_for_en(factor12, col, '12.csv', 'special day')

# 内外利差类 13 14 15

In [14]:
factor13 = loading_file(14)
factor13

Loading file 15 in 22 ...


,中国-美国:10年国债收益率,中国-德国:10年国债收益率,中国-法国:10年国债收益率,中国-英国:10年国债收益率
描述,,,,
1953-04-30,NaN,NaN,NaN,NaN
1953-05-31,NaN,NaN,NaN,NaN
1953-06-30,NaN,NaN,NaN,NaN
1953-07-31,NaN,NaN,NaN,NaN
1953-08-31,NaN,NaN,NaN,NaN
1953-09-30,NaN,NaN,NaN,NaN
1953-10-31,NaN,NaN,NaN,NaN
1953-11-30,NaN,NaN,NaN,NaN
1953-12-31,NaN,NaN,NaN,NaN


In [13]:
adf = ADFtest(factor13)
coin = cointest(factor13, 'day')
results = reg(factor13, 'day')
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,中国-美国：1年国债收益率,-11.818184,-3.438340,-2.865067,平稳
1,中国-德国：1年国债收益率,-6.907966,-3.438272,-2.865037,平稳
2,中国-法国：1年国债收益率,-11.257776,-3.438244,-2.865024,平稳
3,中国-美国：10年国债收益率,-31.887153,-3.438311,-2.865054,平稳
4,中国-德国：10年国债收益率,-29.751618,-3.438187,-2.864999,平稳
5,中国-法国：10年国债收益率,-29.096230,-3.438215,-2.865012,平稳
6,中国-英国：10年国债收益率,-30.587700,-3.438234,-2.865020,平稳


In [14]:
coin

,指标变化率的时间序列,P值,是否协整
0,中国-美国：1年国债收益率,0.889138,否
1,中国-德国：1年国债收益率,0.308716,否
2,中国-法国：1年国债收益率,0.353625,否
3,中国-美国：10年国债收益率,0.326683,否
4,中国-德国：10年国债收益率,0.078291,否
5,中国-法国：10年国债收益率,0.298119,否
6,中国-英国：10年国债收益率,0.148505,否


In [15]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,中国-美国：1年国债收益率,0.311164,4.193637e-16,0.077178,826
1,中国-德国：1年国债收益率,0.785700,0.000000e+00,0.912093,839
2,中国-法国：1年国债收益率,0.838363,0.000000e+00,0.927534,836
3,中国-美国：10年国债收益率,0.662130,1.853088e-26,0.128539,826
4,中国-德国：10年国债收益率,1.464513,8.618984e-200,0.662828,839
5,中国-法国：10年国债收益率,1.286569,1.387113e-179,0.624528,836
6,中国-英国：10年国债收益率,0.933601,6.801863e-203,0.670725,834


In [15]:
factor13.columns

Index(['中国-美国:10年国债收益率', '中国-德国:10年国债收益率', '中国-法国:10年国债收益率', '中国-英国:10年国债收益率'], dtype='object')

In [18]:
col1 = ['中国-美国：1年国债收益率', '中国-德国：1年国债收益率', '中国-法国：1年国债收益率', '中国-美国：10年国债收益率',
       '中国-德国：10年国债收益率', '中国-法国：10年国债收益率', '中国-英国：10年国债收益率']
csv_for_en(factor13, col, '13.csv', 'day')

# 汇率类 16

In [42]:
factor14 = loading_file(15)
factor14

Loading file 16 in 22 ...


,美元指数,欧元兑美元,美元兑日元,英镑兑美元,美元兑人民币元
描述,,,,,
1971-01-04,120.5300,NaN,357.73,2.3938,NaN
1971-01-05,120.5200,NaN,357.81,2.3949,NaN
1971-01-06,120.4900,NaN,357.86,2.3967,NaN
1971-01-07,120.5500,NaN,357.87,2.3963,NaN
1971-01-08,120.5300,NaN,357.82,2.3972,NaN
1971-01-11,120.5300,NaN,357.95,2.3992,NaN
1971-01-12,120.4900,NaN,358.06,2.4001,NaN
1971-01-13,120.4700,NaN,358.44,2.4021,NaN
1971-01-14,120.4400,NaN,358.40,2.4071,NaN


In [43]:
coin = cointest(factor14, 'day')
results = reg(factor14, 'day')
coin

,指标变化率的时间序列,P值,是否协整
0,美元指数,0.062814,否
1,欧元兑美元,0.151830,否
2,美元兑日元,0.446895,否
3,英镑兑美元,0.816130,否
4,美元兑人民币元,0.662470,否


In [44]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,美元指数,-0.020643,2.167811e-66,0.071171,4015
1,欧元兑美元,1.370499,3.095645e-62,0.069057,3876
2,美元兑日元,-0.005702,2.412775e-11,0.011449,3876
3,英镑兑美元,0.198947,7.866161e-04,0.002906,3876
4,美元兑人民币元,-0.362544,3.262413e-145,0.156286,3877


In [23]:
col = ['美元指数', '欧元兑美元', '美元兑日元', '英镑兑美元', '美元兑人民币元']
csv_for_en(factor14, col, '14.csv', 'day')

# 跨境资本流动 17

In [45]:
factor15 = loading_file(16)
factor15

Loading file 17 in 22 ...


,官方储备资产:外汇储备,货币当局:国外资产:外汇(中央银行外汇占款),金融机构:人民币:资金运用:中央银行外汇占款
描述,,,
1950-12-01,1.57,NaN,NaN
1951-12-01,0.45,NaN,NaN
1952-12-01,1.08,NaN,NaN
1953-12-01,0.90,NaN,NaN
1954-12-01,0.88,NaN,NaN
1955-12-01,1.80,NaN,NaN
1956-12-01,1.17,NaN,NaN
1957-12-01,1.23,NaN,NaN
1958-12-01,0.70,NaN,NaN


In [46]:
adf = ADFtest(factor15)
coin = cointest(factor15, 'month')
results = reg(factor15, 'month')
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,官方储备资产:外汇储备,-2.648792,-3.450201,-2.870285,非平稳
1,货币当局:国外资产:外汇(中央银行外汇占款),-2.347478,-3.460708,-2.874891,非平稳
2,金融机构:人民币:资金运用:中央银行外汇占款,-1.356640,-3.711212,-2.981247,非平稳


In [26]:
coin

,指标变化率的时间序列,P值,是否协整
0,官方储备资产:外汇储备,0.695424,否
1,货币当局:国外资产:外汇(中央银行外汇占款),0.317403,否
2,金融机构:人民币:资金运用:中央银行外汇占款,0.738920,否


In [47]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,官方储备资产:外汇储备,0.000026,4.075730e-11,0.199797,198
1,货币当局:国外资产:外汇(中央银行外汇占款),0.000003,1.177801e-09,0.173250,197
2,金融机构:人民币:资金运用:中央银行外汇占款,-0.000048,1.470149e-08,0.701372,29


In [29]:
col = ['官方储备资产:外汇储备', '货币当局:国外资产:外汇(中央银行外汇占款)']
csv_for_en(factor15, col, '15.csv', 'month')

# 财政政策类 18

In [48]:
factor16 = loading_file(17)
factor16

Loading file 18 in 22 ...


,债券发行量:政府债券:累计值,债券发行量:政策性银行债:累计值,债券发行量:地方政府债:累计值,公共财政支出:累计同比,公共财政支出:当月同比,公共财政收入:累计同比,公共财政收入:当月同比
描述,,,,,,,
1990-01-01,NaN,NaN,NaN,10.10,10.10,-4.20,-4.20
1990-02-01,NaN,NaN,NaN,23.60,40.40,-1.10,2.00
1990-03-01,NaN,NaN,NaN,17.00,8.50,4.70,13.00
1990-04-01,NaN,NaN,NaN,14.10,8.20,5.60,7.80
1990-05-01,NaN,NaN,NaN,11.50,4.20,9.00,21.90
1990-06-01,NaN,NaN,NaN,12.20,14.40,10.50,16.50
1990-07-01,NaN,NaN,NaN,16.40,36.90,11.60,16.40
1990-08-01,NaN,NaN,NaN,18.00,27.40,13.00,21.60
1990-09-01,NaN,NaN,NaN,16.80,9.00,13.70,18.40


In [49]:
adf = ADFtest(factor16)
coin = cointest(factor16, 'month')
results = reg(factor16, 'month')
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,债券发行量:政府债券:累计值,-3.759026,-3.457551,-2.873509,平稳
1,债券发行量:政策性银行债:累计值,-5.753837,-3.458011,-2.873710,平稳
2,债券发行量:地方政府债:累计值,-2.323418,-3.498198,-2.891208,非平稳
3,公共财政支出:累计同比,-8.649146,-3.450569,-2.870447,平稳
4,公共财政支出:当月同比,-8.322982,-3.450569,-2.870447,平稳
5,公共财政收入:累计同比,-9.222124,-3.451349,-2.870789,平稳
6,公共财政收入:当月同比,-9.185558,-3.450695,-2.870502,平稳


In [32]:
coin

,指标变化率的时间序列,P值,是否协整
0,债券发行量:政府债券:累计值,0.895370,否
1,债券发行量:政策性银行债:累计值,0.851931,否
2,债券发行量:地方政府债:累计值,0.895834,否
3,公共财政支出:累计同比,0.456405,否
4,公共财政支出:当月同比,0.652840,否
5,公共财政收入:累计同比,0.705435,否
6,公共财政收入:当月同比,0.622559,否


In [50]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,债券发行量:政府债券:累计值,7.254413e-06,1.090928e-02,0.032602,198
1,债券发行量:政策性银行债:累计值,3.367444e-05,1.280339e-07,0.132972,198
2,债券发行量:地方政府债:累计值,-1.144637e-07,9.807534e-01,0.000005,112
3,公共财政支出:累计同比,-4.271742e-03,4.129719e-01,0.003440,197
4,公共财政支出:当月同比,-4.716105e-03,1.694041e-01,0.009661,197
5,公共财政收入:累计同比,1.027125e-02,2.790975e-02,0.024797,195
6,公共财政收入:当月同比,-3.661472e-03,3.651344e-01,0.004208,197


In [34]:
col = ['公共财政收入:当月同比', '公共财政支出:当月同比', '公共财政支出:累计同比', '公共财政收入:累计同比', 
       '债券发行量:政府债券:累计值', '债券发行量:地方政府债:累计值', '债券发行量:政策性银行债:累计值']
csv_for_en(factor16, col, '16.csv', 'month')

# 货币政策类 19

In [10]:
factor17 = loading_file(18)
factor17

Loading file 19 in 22 ...


,中期借贷便利(MLF):投放数量:6个月,中期借贷便利(MLF):投放数量:1年,抵押补充贷款(PSL):期末余额,抵押补充贷款(PSL):提供资金:当月新增,公开市场操作:货币净投放,公开市场操作:货币投放,公开市场操作:货币回笼,逆回购:到期量,人民币存款准备金率:中小型存款类金融机构(月),人民币存款准备金率:大型存款类金融机构(月),超额存款准备金率(超储率):金融机构,超额存款准备金率(超储率):农村信用社,货币当局:国外资产:外汇(中央银行外汇占款),货币当局:对其他存款性公司债权,货币当局:政府存款
描述,,,,,,,,,,,,,,,
1985-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN
1985-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,NaN,NaN


In [11]:
adf = ADFtest(factor17)
coin = cointest(factor17, 'day')
results = reg(factor17, 'day')
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,中期借贷便利(MLF):投放数量:6个月,-9.275465,-3.679060,-2.967882,平稳
1,中期借贷便利(MLF):投放数量:1年,-7.589415,-3.639224,-2.951230,平稳
2,抵押补充贷款(PSL):期末余额,-6.355193,-3.626652,-2.945951,平稳
3,抵押补充贷款(PSL):提供资金:当月新增,-2.029191,-3.711212,-2.981247,非平稳
4,公开市场操作:货币净投放,-11.139513,-3.439686,-2.865660,平稳
5,公开市场操作:货币投放,-9.462743,-3.440104,-2.865844,平稳
6,公开市场操作:货币回笼,-8.499164,-3.440207,-2.865889,平稳
7,逆回购:到期量,-8.606303,-3.460849,-2.874953,平稳
8,人民币存款准备金率:中小型存款类金融机构(月),-8.960974,-3.446888,-2.868829,平稳
9,人民币存款准备金率:大型存款类金融机构(月),-9.107425,-3.446888,-2.868829,平稳


In [43]:
coin

,指标变化率的时间序列,P值,是否协整
0,中期借贷便利(MLF):投放数量:6个月,0.834500,否
1,中期借贷便利(MLF):投放数量:1年,0.952154,否
2,抵押补充贷款(PSL):期末余额,0.959161,否
3,抵押补充贷款(PSL):提供资金:当月新增,0.792554,否
4,公开市场操作:货币净投放,0.001600,是
5,公开市场操作:货币投放,0.456920,否
6,公开市场操作:货币回笼,0.371368,否
7,逆回购:到期量,0.371368,否
8,人民币存款准备金率:中小型存款类金融机构(月),0.993927,否
9,人民币存款准备金率:大型存款类金融机构(月),0.993927,否


In [12]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,中期借贷便利(MLF):投放数量:6个月,-0.000230,5.739856e-02,0.118966,31
1,中期借贷便利(MLF):投放数量:1年,0.000300,9.139552e-08,0.562456,37
2,抵押补充贷款(PSL):期末余额,0.000072,1.197291e-07,0.666091,28
3,抵押补充贷款(PSL):提供资金:当月新增,-0.000025,8.831155e-01,0.000882,27
4,公开市场操作:货币净投放,0.000013,4.250454e-01,0.000926,689
5,公开市场操作:货币投放,0.000020,1.615432e-01,0.002963,663
6,公开市场操作:货币回笼,0.000005,7.266353e-01,0.000186,661
7,逆回购:到期量,-0.000011,4.615806e-01,0.002560,214
8,人民币存款准备金率:中小型存款类金融机构(月),0.080586,2.514673e-10,0.244830,145
9,人民币存款准备金率:大型存款类金融机构(月),0.062105,9.684803e-09,0.206169,145


# 其他政府监管类 20

In [13]:
factor18 = loading_file(19)
factor18

Loading file 20 in 22 ...


,1年期国债收益率-SHIBOR:3个月,1年期国债收益率-FR007,1年期国债收益率-GC007,信托业:未来一年到期规模:合计,信托业:未来一年到期规模:集合信托,信托业:未来一年到期规模:单一信托,信托业:未来一年到期规模:财产权信托,保险公司:银行存款和债券:占资金运用余额比例,保险公司:保险资金运用余额:投资:债券投资,基金管理公司管理资产规模:公募基金,...,公募基金净值:开放式基金:债券型,证券投资基金成交金额:当月值,银行理财产品资金余额,中债:债券托管量:国债:商业银行,中债:债券托管量:国债:保险机构,待购回债券余额,中债:债券托管量:国债:合计,债券市场发行债券:同业存单,债券市场托管余额:同业存单,其他存款性公司:对其他金融机构债权
描述,,,,,,,,,,,,,,,,,,,,,
1997-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,159.03,0.00,0.00,184.49,NaN,NaN,NaN
1997-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,197.67,0.00,0.00,226.71,NaN,NaN,NaN
1997-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,306.25,0.00,0.00,335.08,NaN,NaN,NaN
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,357.39,0.00,0.00,973.74,NaN,NaN,NaN
1997-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,376.00,0.00,0.00,975.96,NaN,NaN,NaN
1997-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,406.50,0.00,0.00,1004.91,NaN,NaN,NaN
1997-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,573.16,0.00,0.00,1169.86,NaN,NaN,NaN
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,574.41,0.00,0.00,1168.78,NaN,NaN,NaN


In [14]:
adf = ADFtest(factor18)
coin = cointest(factor18, 'day', part = True, cols = ['1年期国债收益率-SHIBOR:3个月', '1年期国债收益率-FR007', '1年期国债收益率-GC007'])
results = reg(factor18, 'day', part = True, cols = ['1年期国债收益率-SHIBOR:3个月', '1年期国债收益率-FR007', '1年期国债收益率-GC007'])
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,1年期国债收益率-SHIBOR:3个月,-19.237306,-3.432576,-2.862524,平稳
1,1年期国债收益率-FR007,-15.607590,-3.431947,-2.862246,平稳
2,1年期国债收益率-GC007,-16.402068,-3.432572,-2.862522,平稳
3,信托业:未来一年到期规模:合计,-4.241494,-3.596636,-2.933297,平稳
4,信托业:未来一年到期规模:集合信托,-1.769253,-3.592504,-2.931550,非平稳
5,信托业:未来一年到期规模:单一信托,-4.675879,-3.596636,-2.933297,平稳
6,信托业:未来一年到期规模:财产权信托,-3.390726,-3.596636,-2.933297,非平稳
7,保险公司:银行存款和债券:占资金运用余额比例,-1.698282,-3.540523,-2.909427,非平稳
8,保险公司:保险资金运用余额:投资:债券投资,-4.807629,-3.518281,-2.899878,平稳
9,基金管理公司管理资产规模:公募基金,-12.147137,-3.517114,-2.899375,平稳


In [66]:
coin

,指标变化率的时间序列,P值,是否协整
0,1年期国债收益率-SHIBOR:3个月,0.054807,否
1,1年期国债收益率-FR007,0.032472,是
2,1年期国债收益率-GC007,0.005955,是


In [15]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,1年期国债收益率-SHIBOR:3个月,-0.592360,9.287945e-170,0.230589,2945
1,1年期国债收益率-FR007,-0.025375,7.955341e-02,0.000745,4128
2,1年期国债收益率-GC007,-0.029920,4.055802e-07,0.008606,2972


In [16]:
col = ['信托业:未来一年到期规模:合计', '信托业:未来一年到期规模:集合信托', '信托业:未来一年到期规模:单一信托',
       '信托业:未来一年到期规模:财产权信托', '保险公司:银行存款和债券:占资金运用余额比例', '保险公司:保险资金运用余额:投资:债券投资',
       '基金管理公司管理资产规模:公募基金', '公募基金份额:开放式基金:债券型', '公募基金净值:开放式基金:债券型',
       '证券投资基金成交金额:当月值', '银行理财产品资金余额', '中债:债券托管量:国债:商业银行', '中债:债券托管量:国债:保险机构',
       '待购回债券余额', '中债:债券托管量:国债:合计', '债券市场发行债券:同业存单', '债券市场托管余额:同业存单',
       '其他存款性公司:对其他金融机构债权']
coin = cointest(factor18, 'month', part = True, cols = col)
results = reg(factor18, 'month', part = True, cols = col)
coin

,指标变化率的时间序列,P值,是否协整
0,信托业:未来一年到期规模:合计,0.839503,否
1,信托业:未来一年到期规模:集合信托,0.915602,否
2,信托业:未来一年到期规模:单一信托,0.406578,否
3,信托业:未来一年到期规模:财产权信托,0.987920,否
4,保险公司:银行存款和债券:占资金运用余额比例,0.892081,否
5,保险公司:保险资金运用余额:投资:债券投资,0.992791,否
6,基金管理公司管理资产规模:公募基金,0.990380,否
7,公募基金份额:开放式基金:债券型,0.982292,否
8,公募基金净值:开放式基金:债券型,0.982990,否
9,证券投资基金成交金额:当月值,0.314625,否


In [17]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,信托业:未来一年到期规模:合计,1.170363e-04,6.530785e-02,0.075114,46
1,信托业:未来一年到期规模:集合信托,4.581980e-04,2.803755e-02,0.104988,46
2,信托业:未来一年到期规模:单一信托,8.705828e-05,3.492236e-01,0.019941,46
3,信托业:未来一年到期规模:财产权信托,9.259338e-04,4.339732e-03,0.170545,46
4,保险公司:银行存款和债券:占资金运用余额比例,-6.784239e-03,3.245110e-01,0.013119,76
5,保险公司:保险资金运用余额:投资:债券投资,1.736167e-09,5.798962e-03,0.093540,80
6,基金管理公司管理资产规模:公募基金,1.025921e-06,6.359924e-01,0.003043,76
7,公募基金份额:开放式基金:债券型,-9.666794e-07,9.485265e-01,0.000066,66
8,公募基金净值:开放式基金:债券型,-6.255370e-06,6.623210e-01,0.002999,66
9,证券投资基金成交金额:当月值,-1.586775e-06,9.568206e-01,0.000015,194


# 资金策略类 21

In [52]:
factor19 = loading_file(20)
factor19

Loading file 21 in 22 ...


,1年期国债收益率-中债国开债到期收益率:1年,1年期国债收益率-中债商业银行普通债到期收益率(AAA):1年,1年期国债收益率-同业存单到期收益率(AAA+):1年,1年期国债收益率-短期融资券到期收益率(AAA+):1年,1年期国债收益率-中债企业债到期收益率(AAA):1年,1年期国债收益率-中债铁道债到期收益率:1年,1年期国债收益率-中债地方政府债到期收益率(AAA):1年,1年期国债收益率-贷款基础利率(LPR):1年,居民活期存款/各项贷款,居民活期存款/居住中长期贷款,居民活期存款/非金融企业及机关团体各项贷款,非金融企业活期存款/各项贷款,非金融企业活期存款/居住中长期贷款,非金融企业活期存款/非金融企业及机关团体各项贷款,债券投资/资金运用总计,各项贷款/债券投资
描述,,,,,,,,,,,,,,,,
1978-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
factor19['非金融企业活期存款/非金融企业及机关团体各项贷款'].dropna()

描述
2011-01-31    1.421111
2011-02-28    1.443652
2011-03-31    1.443115
2011-04-30    1.431888
2011-05-31    1.418480
2011-06-30    1.391243
2011-07-31    1.346073
2011-08-31    1.321707
2011-09-30    1.228482
2011-10-31    1.292959
2011-11-30    1.302392
2011-12-31    1.336152
2012-01-31    1.141804
2012-02-29    1.152014
2012-03-31    1.143873
2012-04-30    1.109154
2012-05-31    1.123839
2012-06-30    1.120691
2012-07-31    1.079495
2012-08-31    1.068067
2012-09-30    1.024431
2012-10-31    1.062215
2012-11-30    1.077877
2012-12-31    1.117804
2013-01-31    1.071800
2013-02-28    1.019065
2013-03-31    1.044480
2013-04-30    1.011111
2013-05-31    0.995160
2013-06-30    0.970380
                ...   
2015-12-31    0.883547
2016-01-31    0.869585
2016-02-29    0.825980
2016-03-31    0.863858
2016-04-30    0.854314
2016-05-31    0.851779
2016-06-30    0.878270
2016-07-31    0.860725
2016-08-31    0.874518
2016-09-30    0.863045
2016-10-31    0.885011
2016-11-30    0.903324
2016-12-

In [53]:
adf = ADFtest(factor19)
col1 = ['1年期国债收益率-中债国开债到期收益率:1年',
 '1年期国债收益率-中债商业银行普通债到期收益率(AAA):1年',
 '1年期国债收益率-同业存单到期收益率(AAA+):1年',
 '1年期国债收益率-短期融资券到期收益率(AAA+):1年',
 '1年期国债收益率-中债企业债到期收益率(AAA):1年',
 '1年期国债收益率-中债铁道债到期收益率:1年',
 '1年期国债收益率-中债地方政府债到期收益率(AAA):1年',
 '1年期国债收益率-贷款基础利率(LPR):1年']
col2 = ['居民活期存款/各项贷款',
 '居民活期存款/居住中长期贷款',
 '居民活期存款/非金融企业及机关团体各项贷款',
 '非金融企业活期存款/各项贷款',
 '非金融企业活期存款/居住中长期贷款',
 '非金融企业活期存款/非金融企业及机关团体各项贷款',
 '债券投资/资金运用总计',
 '各项贷款/债券投资']
coin = cointest(factor19, 'day', part = True, cols = col1)
results = reg(factor19, 'day', part = True, cols = col1)
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,1年期国债收益率-中债国开债到期收益率:1年,-14.631918,-3.431948,-2.862246,平稳
1,1年期国债收益率-中债商业银行普通债到期收益率(AAA):1年,-22.086297,-3.433411,-2.862892,平稳
2,1年期国债收益率-同业存单到期收益率(AAA+):1年,-20.900597,-3.436079,-2.864070,平稳
3,1年期国债收益率-短期融资券到期收益率(AAA+):1年,-39.575627,-3.434441,-2.863347,平稳
4,1年期国债收益率-中债企业债到期收益率(AAA):1年,-27.796225,-3.432463,-2.862474,平稳
5,1年期国债收益率-中债铁道债到期收益率:1年,-24.541738,-3.433006,-2.862713,平稳
6,1年期国债收益率-中债地方政府债到期收益率(AAA):1年,-14.481139,-3.433672,-2.863008,平稳
7,1年期国债收益率-贷款基础利率(LPR):1年,-10.422886,-3.435928,-2.864003,平稳
8,居民活期存款/各项贷款,-2.794598,-3.480500,-2.883528,非平稳
9,居民活期存款/居住中长期贷款,-7.048787,-3.605565,-2.937069,平稳


In [75]:
coin

,指标变化率的时间序列,P值,是否协整
0,1年期国债收益率-中债国开债到期收益率:1年,0.011587,是
1,1年期国债收益率-中债商业银行普通债到期收益率(AAA):1年,0.021759,是
2,1年期国债收益率-同业存单到期收益率(AAA+):1年,0.096774,否
3,1年期国债收益率-短期融资券到期收益率(AAA+):1年,0.027599,是
4,1年期国债收益率-中债企业债到期收益率(AAA):1年,0.052055,否
5,1年期国债收益率-中债铁道债到期收益率:1年,0.055330,否
6,1年期国债收益率-中债地方政府债到期收益率(AAA):1年,0.166828,否
7,1年期国债收益率-贷款基础利率(LPR):1年,0.004174,是


In [54]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,1年期国债收益率-中债国开债到期收益率:1年,-0.513082,3.514820e-62,0.064937,4127
1,1年期国债收益率-中债商业银行普通债到期收益率(AAA):1年,-0.771442,6.907787e-105,0.198453,2141
2,1年期国债收益率-同业存单到期收益率(AAA+):1年,-0.644671,1.978938e-46,0.164027,1145
3,1年期国债收益率-短期融资券到期收益率(AAA+):1年,-0.795053,2.611064e-114,0.276048,1601
4,1年期国债收益率-中债企业债到期收益率(AAA):1年,-0.844429,2.557778e-138,0.183234,3099
5,1年期国债收益率-中债铁道债到期收益率:1年,-1.091362,3.614224e-232,0.349140,2467
6,1年期国债收益率-中债地方政府债到期收益率(AAA):1年,-1.256464,2.413143e-43,0.091861,1980
7,1年期国债收益率-贷款基础利率(LPR):1年,0.467157,2.159070e-70,0.234295,1181


In [55]:
coin = cointest(factor19, 'month', part = True, cols = col2)
results = reg(factor19, 'month', part = True, cols = col2)
coin

,指标变化率的时间序列,P值,是否协整
0,居民活期存款/各项贷款,0.801026,否
1,居民活期存款/居住中长期贷款,0.916445,否
2,居民活期存款/非金融企业及机关团体各项贷款,0.293622,否
3,非金融企业活期存款/各项贷款,0.815404,否
4,非金融企业活期存款/居住中长期贷款,0.909621,否
5,非金融企业活期存款/非金融企业及机关团体各项贷款,0.625336,否
6,债券投资/资金运用总计,0.877148,否
7,各项贷款/债券投资,0.816549,否


In [56]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,居民活期存款/各项贷款,-7.432460,0.026301,0.033583,147
1,居民活期存款/居住中长期贷款,-2.900835,0.000003,0.428356,42
2,居民活期存款/非金融企业及机关团体各项贷款,18.978287,0.000006,0.131371,147
3,非金融企业活期存款/各项贷款,2.218556,0.260536,0.014363,90
4,非金融企业活期存款/居住中长期贷款,-5.478045,0.000004,0.415237,42
5,非金融企业活期存款/非金融企业及机关团体各项贷款,2.227927,0.206047,0.018108,90
6,债券投资/资金运用总计,-1.065874,0.847077,0.000425,90
7,各项贷款/债券投资,0.125469,0.180566,0.020282,90


In [79]:
csv_for_en(factor19, col1, '19_1.csv', 'day')
csv_for_en(factor19, col2, '19_2.csv', 'month')

# 行业监管类 22

In [57]:
factor20 = loading_file(21)
factor20

Loading file 22 in 22 ...


,商业银行:流动性比例,商业银行:存贷比,商业银行:人民币超额备付金率,商业银行:资本充足率,商业银行:净息差,不良贷款余额:商业银行,商业银行:拨备覆盖率
描述,,,,,,,
2002-12-01,NaN,NaN,NaN,NaN,NaN,22793.00,6.90
2003-12-01,NaN,NaN,NaN,NaN,NaN,21044.60,19.70
2004-03-01,NaN,NaN,NaN,NaN,NaN,20776.00,NaN
2004-06-01,NaN,NaN,NaN,NaN,NaN,16631.00,NaN
2004-09-01,NaN,NaN,NaN,NaN,NaN,16998.00,NaN
2004-12-01,NaN,NaN,NaN,NaN,NaN,17175.60,14.20
2005-03-01,NaN,NaN,NaN,NaN,NaN,18274.50,NaN
2005-06-01,NaN,NaN,NaN,NaN,NaN,12759.40,NaN
2005-09-01,NaN,NaN,NaN,NaN,NaN,12808.30,NaN


In [58]:
adf = ADFtest(factor20)
coin = cointest(factor20, 'quater')
results = reg(factor20, 'quater')
adf

,时间序列(差分处理),ADF值,1%临界值,5%临界值,是否平稳
0,商业银行:流动性比例,-8.006932,-3.626652,-2.945951,平稳
1,商业银行:存贷比,-4.794304,-3.688926,-2.971989,平稳
2,商业银行:人民币超额备付金率,-1.367837,-3.832603,-3.031227,非平稳
3,商业银行:资本充足率,-3.095474,-3.711212,-2.981247,非平稳
4,商业银行:净息差,-2.089056,-3.737709,-2.992216,非平稳
5,不良贷款余额:商业银行,-7.358125,-3.550670,-2.913766,平稳
6,商业银行:拨备覆盖率,-2.366606,-3.600983,-2.935135,非平稳


In [82]:
coin

,指标变化率的时间序列,P值,是否协整
0,商业银行:流动性比例,0.9859,否
1,商业银行:存贷比,0.9859,否
2,商业银行:人民币超额备付金率,0.9859,否
3,商业银行:资本充足率,0.9859,否
4,商业银行:净息差,0.9859,否
5,不良贷款余额:商业银行,0.9859,否
6,商业银行:拨备覆盖率,0.9859,否


In [59]:
results

,指标变化率的时间序列,coef,Prob(F-statistic),R-squared,No.Observations
0,商业银行:流动性比例,0.071777,0.140548,0.059338,38
1,商业银行:存贷比,0.033525,0.582957,0.010901,30
2,商业银行:人民币超额备付金率,-0.077072,0.722043,0.004590,30
3,商业银行:资本充足率,0.369795,0.025921,0.133909,37
4,商业银行:净息差,-0.122027,0.801757,0.002289,30
5,不良贷款余额:商业银行,0.000004,0.851481,0.000620,59
6,商业银行:拨备覆盖率,0.004376,0.002021,0.205129,44


In [61]:
col = ['商业银行:流动性比例', '商业银行:存贷比', '商业银行:人民币超额备付金率', '商业银行:资本充足率', '商业银行:净息差',
       '不良贷款余额:商业银行', '商业银行:拨备覆盖率']
csv_for_en(factor20, col, 'new.csv', 'quater')